This notebook is responsible for doing the following:

1. Getting the context values for each airport
2. Computing the ranking and classifications
3. Construct H and G sets
4. Running LLMs
5. Voting function

## Generating context of nodes

In [ ]:
# Getting the gerenated context for all Airports entities

import pandas as pd
import ast

dfEntitiesContext = pd.read_csv("../context_entities.csv")
dfEntitiesContext["value"] = dfEntitiesContext["value"].apply(ast.literal_eval)

dfEntitiesContext.head()

       key                                              value
0     KATL  [[node, [], Airport, tz_database_timezone, Ame...
1     KORD  [[node, [], Airport, tz_database_timezone, Ame...
2     KLAX  [[node, [], Airport, tz_database_timezone, Ame...
3     KDFW  [[node, [], Airport, tz_database_timezone, Ame...
4     LFPG  [[node, [], Airport, tz_database_timezone, Eur...
...    ...                                                ...
3209  NIUE  [[node, [], Airport, tz_database_timezone, Pac...
3210  YPOD  [[node, [], Airport, tz_database_timezone, Aus...
3211  PAIG  [[node, [], Airport, tz_database_timezone, Ame...
3212  PAPH  [[node, [], Airport, tz_database_timezone, Ame...
3213  LTCM  [[node, [], Airport, tz_database_timezone, Eur...

[3214 rows x 2 columns]


,key,value
0,KATL,"[[node, [], Airport, tz_database_timezone, Ame..."
1,KORD,"[[node, [], Airport, tz_database_timezone, Ame..."
2,KLAX,"[[node, [], Airport, tz_database_timezone, Ame..."
3,KDFW,"[[node, [], Airport, tz_database_timezone, Ame..."
4,LFPG,"[[node, [], Airport, tz_database_timezone, Eur..."


In [ ]:
#  Function for getting the values from an specific airport

def getEntityValuesFromContext(entity):
    props = {}
    row = dfEntitiesContext[dfEntitiesContext['key']==entity]['value'].to_list()
    for values in row[0]:
        if values[0] == 'node' and values[1]==[]:
            props[values[3]] = values[4]

    contextLeft = row[0][len(props):]
    return props, contextLeft

## Running the protocol

1. Getting the rankings and classifications
2. Choosing random properties
3. Constructing the H and G subsets with randomized properties

In [ ]:
import random
import pandas as pd 


# Classification of properties based on their popularity
propertiesRanking = pd.read_csv('../datasets/ranks/rank_properties.csv')
propertiesClassification = {'easy': propertiesRanking[propertiesRanking['value']>=80]['prop'].to_list(),
'hard': propertiesRanking[propertiesRanking['value']<80]['prop'].to_list()
}

# Function for choosing random properties from a category
def chooseRandomProperties(n, category, available=[]):

    category_props = set(propertiesClassification[category])  
    available_props = set(available)
    
    if available!=[]:
        candidates = list(category_props & available_props)
    else:
        candidates = propertiesClassification[category]

    return random.sample(candidates, n)

# Function that constructs each subset --  1 easy easy, 1 easy hard, 1 hard easy, 1 hard hard - random combination of properties
def constructSubsets(available):
    available = available

    # Helper function auxiliar to generate H and G
    def generate_hidden_given(hidden_difficulty, given_difficulty, available):

        hidden = chooseRandomProperties(3, hidden_difficulty)
        given_candidates = chooseRandomProperties(3, given_difficulty, available)

        given = [prop for prop in given_candidates if prop not in hidden]

        while len(given) < 2:
            new_prop = chooseRandomProperties(1, given_difficulty)[0]
            if new_prop not in hidden and new_prop not in given:
             given.append(new_prop)

        return hidden, given

    easy_easyHidden, easy_easyGiven = generate_hidden_given("easy", "easy",available)
    easy_hardHidden, easy_hardGiven = generate_hidden_given("easy", "hard",available)
    hard_easyHidden, hard_easyGiven = generate_hidden_given("hard", "easy",available)
    hard_hardHidden, hard_hardGiven = generate_hidden_given("hard", "hard",available)

    subsetsTest = {
        'easyeasy': [easy_easyHidden, easy_easyGiven],
        'easyhard': [easy_hardHidden, easy_hardGiven],
        'hardeasy': [hard_easyHidden, hard_easyGiven],
        'hardhard': [hard_hardHidden, hard_hardGiven]
    }

    return subsetsTest


In [ ]:
import pandas as pd
import random

# Classification of entities based on their PageRank score

entities = pd.read_csv('../datasets/ranks/ranks_pagerank.csv')
entities["quartile"] = pd.qcut(entities["score"], q=3, labels=[1, 2, 3])

entitiesCat = {
    'easy': entities[entities["quartile"]==3]['icao'].to_list(),
    'medium': entities[entities["quartile"]==2]['icao'].to_list(),
    'hard': entities[entities["quartile"]==1]['icao'].to_list(),

}

# Functions that choose random entities from a category
def chooseRandomEntities(n,category):
    return random.sample(entitiesCat[category], n)

# Generating the nodes for the tests (3 in each category)
def generateNodes(n=3):
    easyNodes = chooseRandomEntities(n,"easy")
    mediumNodes = chooseRandomEntities(n,"medium")
    hardNodes = chooseRandomEntities(n,"hard")

    return {'easy': easyNodes,'medium': mediumNodes, 'hard': hardNodes}

# Generate the tasks -- These will be passed to the LLM for evaluation. Each task is a tuple with the following format: (entity, hidden properties, given properties, metadata)
def generateTasks():
    
    listOfTasks = []

    for diff, entityClass in generateNodes().items():
        for entity in entityClass:
            entityProps, entityContext = getEntityValuesFromContext(entity)
            subsets = constructSubsets(available=list(entityProps.keys()))
            for subset in subsets.items():
                metadata = {'difficultyProp':subset[0],"entityDiff:":diff, 'context':entityContext}
                task = (entity, subset[1][0], subset[1][1],metadata)
                listOfTasks.append(task)

    return listOfTasks


resultsTasks = generateTasks()

print(len(resultsTasks))
for x in resultsTasks:
    print(x)
    

36
('PAKK', ['location', 'source', 'dst'], ['altitude', 'city'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'easy', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-165.44500732421875, 64.51219940185547]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 37.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5788.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Nome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAOM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Nome Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 22

In [ ]:
# Computes difficulty for each task, based on the definitons from the paper

import math

def computePropPop(propertyName):
    val = propertiesRanking.loc[propertiesRanking['prop'] == propertyName, 'value'].iloc[0]
   
    percentage = val/100

    return float(percentage)

def computeCentralityNormalized(icao):
    # These are hard coded from the database, but they represent the min and max PageRank scores among all entities, 
    # which allows us to normalize the centrality score of each entity to a 0-1 range.
    min_s = math.log(0.15000000000000002)
    max_s = math.log(29.536791458876174)
    score = math.log(entities.loc[entities['icao'] == icao, 'score'].iloc[0])
    
    normalized = (score - min_s) / (max_s - min_s)
    return normalized

def specificity(entity, properties):
    if not properties:
        return 0.0

    acc = 0.0
    for prop in properties:
        log = math.log(computePropPop(prop) * computeCentralityNormalized(entity))
        acc += log

    
    return math.exp(-acc / len(properties))


def dif(task):
    """
    task = (
        entity_icao,
        H_set,
        G_set,
        metadata
    )
    """
    entity, H, G, meta = task

    spec_H = specificity(entity, H)
    spec_G = specificity(entity, G)
    print(f"Entity {entity} | diffEnt: {meta['entityDiff:']} | diffProps {meta['difficultyProp']} | SpecH {spec_H} | SpecG {spec_G} | diff: {spec_H+spec_G}")

for val in resultsTasks:
    dif(val)

Entity PAKK | diffEnt: easy | diffProps easyeasy | SpecH 3.2841419382280166 | SpecG 3.2946443605314064 | diff: 6.5787862987594234
Entity PAKK | diffEnt: easy | diffProps easyhard | SpecH 3.2911398266239655 | SpecG 4.48568570625963 | diff: 7.776825532883596
Entity PAKK | diffEnt: easy | diffProps hardeasy | SpecH 4.375643336186732 | SpecG 3.2841419382280166 | diff: 7.659785274414748
Entity PAKK | diffEnt: easy | diffProps hardhard | SpecH 4.375643336186732 | SpecG 12.346045282045866 | diff: 16.721688618232598
Entity YPPH | diffEnt: easy | diffProps easyeasy | SpecH 1.4563609085911315 | SpecG 1.4563609085911315 | diff: 2.912721817182263
Entity YPPH | diffEnt: easy | diffProps easyhard | SpecH 1.4563609085911315 | SpecG 1.9891884801871067 | diff: 3.4455493887782382
Entity YPPH | diffEnt: easy | diffProps hardeasy | SpecH 1.9403899175557495 | SpecG 1.5623661803961668 | diff: 3.5027560979519166
Entity YPPH | diffEnt: easy | diffProps hardhard | SpecH 3.810848574755258 | SpecG 1.989188480187

In [ ]:
# This code is responsible for constructing the prompts and inputs to be passed to the LLM. 
import os

folder_path = "../prompts/"

files = os.listdir(folder_path)
texts = []

for filename in files:
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            texts.append(content)

def getValuesFromEntityContext(listOfProps,entity):

    entityVal = getEntityValuesFromContext(entity=entity)[0]
    values = [f'{key}:{value}' for key,value in entityVal.items() if key in listOfProps]
    return ",".join(values)

def constructInputForEntity(entity, given, context=[]):

    baseInput = f"""
    Here is the entity extracted from the database that you should seek information about:

        input: [INPUT]

        This is the context about this entity, its relationships on the graph: [NO_CONTEXT_ADDED].
    """
 
    data = getValuesFromEntityContext(given, entity) 
    inputUser = baseInput.replace('[INPUT]', data)

    if context!=[]:
        contextGenerated = context[:random.randint(4,15)]
        inputUser = inputUser.replace('[NO_CONTEXT_ADDED]', contextGenerated)

    return inputUser

def constructPrompt(hidden):

    listOfPrompts = []
    for prompt in texts:
        promptText = prompt.replace('[PROPERTIES]', ",".join(hidden))
        listOfPrompts.append(promptText)

    return listOfPrompts

def generateInputAndPrompt(data):
    for index, row in enumerate(data):
        entity, hidden, given, metadata = row

        inputUser = constructInputForEntity(entity, given, context)
        promptsForEntity = constructPrompt(hidden)

        data[index][3]['input'] = inputUser
        data[index][3]['prompts'] = promptsForEntity

    return data


results = generateInputAndPrompt(resultsTasks)

In [ ]:
diff_per_task = pd.read_csv("diff_per_task.csv",sep=';')
diff_per_task.head()

,Entity,diffEnt,diffProps,SpecH,SpecG,diff
0,PAKK,easy,easyeasy,"3,2841419382280166","3,2946443605314064","6,5787862987594234"
1,PAKK,easy,easyhard,"3,2911398266239655","4,48568570625963","7,776825532883596"
2,PAKK,easy,hardeasy,"4,375643336186732","3,2841419382280166","7,659785274414748"
3,PAKK,easy,hardhard,"4,375643336186732","12,346045282045866","16,721688618232598"
4,YPPH,easy,easyeasy,"1,4563609085911315","1,4563609085911315","2,912721817182263"


In [350]:

import csv

for (entity, h, g, meta) in results:
    diffprop = meta['difficultyProp']
    diffent = meta['entityDiff:']

    infoTask = diff_per_task[(diff_per_task['Entity']==entity) & (diff_per_task['diffProps']==diffprop)].iloc[0]
    spech = infoTask['SpecH']
    specg = infoTask['SpecG']
    diff = infoTask['diff']

    meta['diffScore'] = diff
    meta['specificityH'] = spech
    meta['specificityG'] = specg


with open("test_results.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for s in results:
        writer.writerow(list(s))

In [ ]:
listAllDetails = []

promptNamesList = ["compact","few5","few3","few1","zero","5cot","zerocot","3cot","1cot"]

for (entity, h, g, meta) in resultsTasks:
    spech = meta['diffScore']
    specg = meta['specificityH']
    diff = meta['specificityG']
    promptsList = meta['prompts']
    inputText = meta['input']
    context = meta['context']
    
    diffPropQuartile = meta['difficultyProp']
    diffEntityQuartile = meta['entityDiff:']
    centralityScore = float(entities[entities['icao']==entity].iloc[0]['score']) # Its not normalized
    centralityScoreNorm = computeCentralityNormalized(entity)
    propsScoreH = [computePropPop(x) for x in h]
    propsScoreG = [computePropPop(x) for x in g]
    acc = 0
    fsc = 0
    
    for index, promptText in enumerate(promptsList):
        promptName = promptNamesList[index]
        newSet = (entity,diffEntityQuartile, centralityScore, centralityScoreNorm,  h, propsScoreH, spech, g, propsScoreG, specg, diffPropQuartile, promptName, promptText, inputText, context, acc, fsc)
        listAllDetails.append(newSet)


print(listAllDetails)
        
    
    
   

In [203]:
def transformResultsToGetContext():
    for result in results:
        metadata = result[3]
        randomContext = metadata['context'][2:random.randint(5,10)]
        metadata['input'] = metadata['input'].replace("[NO_CONTEXT_ADDED]",str(randomContext))
        metadata['input'] = metadata['input'].replace("context:","Here are some relationships and relationships information about this property in the graph:")
        

transformResultsToGetContext()

## Running the experiments for each provider

We will run the necessary code for doing batch requests for each provider

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Read the API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CLAUDE_API_KEY = os.getenv("ANTHROPIC_API_KEY")

### Google

In [ ]:
import json

# This function builds the inline requests to be send to the batch API.
# You need to chance the request_key function accorddingly.


def build_inline_requests(data, n_runs=5):

    requests_by_entity = {}

    for (
        entity_id,
        hidden_props,
        given_props,
        meta
    ) in data:

        entity_input = meta.get("input", "").strip()
        prompts = meta.get("prompts", [])

        entity_requests = []

        for prompt_idx, prompt_text in enumerate(prompts):
            for run in range(n_runs):

                request_key = f"context_gemini25flash_entity_{entity_id}_p{prompt_idx}_run{run+1}_{meta['difficultyProp']}"

                request_payload = {
                    "contents": [
                        {
                            "role": "model",
                            "parts": [
                                {"text": prompt_text.strip().replace("\n", " ")}
                            ]
                        },
                        {
                            "role": "user",
                            "parts": [
                                {"text": entity_input}
                            ]
                        }
                    ]
                }

                entity_requests.append({
                    "key": request_key,
                    "request": request_payload
                })

        requests_by_entity[entity_id] = entity_requests

        with open("gemini_batch_file.jsonl", "a", encoding="utf-8") as f:
            for entity, entity_requests in requests_by_entity.items():
                for item in entity_requests:
                    f.write(json.dumps({
                        "key": item["key"],
                        "request": item["request"]
                    }) + "\n")
    return requests_by_entity

inline = build_inline_requests(results)

In [316]:
results = [('PAKK', ['location', 'source', 'dst'], ['altitude', 'city'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'easy', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-165.44500732421875, 64.51219940185547]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 37.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5788.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Nome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAOM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Nome Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2224.242412712596], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -165.44500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.51219940185547], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3615], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-160.798996, 63.88840103]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 27.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Unalakleet'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'UNK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAUN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Unalakleet Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1790.4116773735], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -160.798996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 63.88840103], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 112.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7088], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-162.2720032, 64.61470032]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 162.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3401.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Elim'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ELI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Elim Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16692328.931118011], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -162.2720032], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.61470032], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 60.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7184], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-161.223999, 64.37110138]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 24.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4001.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shaktoolik'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFSH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shaktoolik Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5345.2476565455145], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -161.223999], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.37110138], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 75.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.0135005732172], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.8401842839167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 118245.43799519724], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: altitude:154.0,city:Koyuk\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: location,source,dst. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: location,source,dst. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('PAKK', ['city', 'longitude', 'source'], ['runwaysLength', 'runways', 'runwaysWidth'], {'difficultyProp': 'easyhard', 'entityDiff:': 'easy', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-165.44500732421875, 64.51219940185547]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 37.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5788.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Nome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAOM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Nome Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2224.242412712596], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -165.44500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.51219940185547], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3615], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-160.798996, 63.88840103]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 27.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Unalakleet'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'UNK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAUN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Unalakleet Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1790.4116773735], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -160.798996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 63.88840103], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 112.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7088], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-162.2720032, 64.61470032]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 162.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3401.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Elim'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ELI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Elim Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16692328.931118011], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -162.2720032], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.61470032], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 60.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7184], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-161.223999, 64.37110138]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 24.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4001.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shaktoolik'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFSH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shaktoolik Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5345.2476565455145], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -161.223999], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.37110138], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 75.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.0135005732172], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.8401842839167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 118245.43799519724], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:3000.0,runways:1.0,runwaysWidth:60.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: city,longitude,source. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,longitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('PAKK', ['runwaysLength', 'runways', 'iata'], ['icao', 'tz_database_timezone', 'longitude'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'easy', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-165.44500732421875, 64.51219940185547]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 37.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5788.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Nome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAOM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Nome Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2224.242412712596], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -165.44500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.51219940185547], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3615], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-160.798996, 63.88840103]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 27.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Unalakleet'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'UNK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAUN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Unalakleet Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1790.4116773735], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -160.798996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 63.88840103], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 112.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7088], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-162.2720032, 64.61470032]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 162.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3401.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Elim'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ELI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Elim Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16692328.931118011], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -162.2720032], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.61470032], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 60.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7184], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-161.223999, 64.37110138]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 24.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4001.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shaktoolik'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFSH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shaktoolik Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5345.2476565455145], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -161.223999], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.37110138], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 75.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.0135005732172], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.8401842839167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 118245.43799519724], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: tz_database_timezone:America/Anchorage,icao:PAKK,longitude:-161.154006958\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runwaysLength,runways,iata. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('PAKK', ['runways', 'iata', 'runwaysLength'], ['runwaysWidth', 'commercial_flights'], {'difficultyProp': 'hardhard', 'entityDiff:': 'easy', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-165.44500732421875, 64.51219940185547]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 37.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5788.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Nome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAOM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Nome Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2224.242412712596], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -165.44500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.51219940185547], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3615], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-160.798996, 63.88840103]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 27.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Unalakleet'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'UNK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PAUN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Unalakleet Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1790.4116773735], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -160.798996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 63.88840103], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 112.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7088], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-162.2720032, 64.61470032]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 162.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3401.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Elim'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ELI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Elim Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16692328.931118011], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -162.2720032], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.61470032], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 60.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7184], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Anchorage'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-161.223999, 64.37110138]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 24.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4001.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shaktoolik'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-9'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'PFSH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shaktoolik Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5345.2476565455145], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -161.223999], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 64.37110138], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 75.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.0135005732172], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.8401842839167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 118245.43799519724], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 209371.21163703914], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 64186.03776495315], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1472'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '8E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 63361.58375425115], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '16726'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'PAG CNC'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '7H']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysWidth:60.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,iata,runwaysLength. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('YPPH', ['source', 'name', 'latitude'], ['icao', 'longitude', 'dst'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'easy', 'context': [['node', ['PASSENGERS'], 'Airport', 'tz_database_timezone', 'Europe/London'], ['node', ['PASSENGERS'], 'Airport', 'commercial_flights', 474576], ['node', ['PASSENGERS'], 'Airport', 'location', [-0.461941, 51.4706]], ['node', ['PASSENGERS'], 'Airport', 'altitude', 83.0], ['node', ['PASSENGERS'], 'Airport', 'dst', 'E'], ['node', ['PASSENGERS'], 'Airport', 'type', 'airport'], ['node', ['PASSENGERS'], 'Airport', 'runwaysLength', 12400.0], ['node', ['PASSENGERS'], 'Airport', 'country', 'United Kingdom'], ['node', ['PASSENGERS'], 'Airport', 'city', 'London'], ['node', ['PASSENGERS'], 'Airport', 'iata', 'LHR'], ['node', ['PASSENGERS'], 'Airport', 'timezone', '0'], ['node', ['PASSENGERS'], 'Airport', 'source', 'OurAirports'], ['node', ['PASSENGERS'], 'Airport', 'runways', 2.0], ['node', ['PASSENGERS'], 'Airport', 'icao', 'EGLL'], ['node', ['PASSENGERS'], 'Airport', 'name', 'London Heathrow Airport'], ['node', ['PASSENGERS'], 'Airport', 'distCity', 25979.20729323409], ['node', ['PASSENGERS'], 'Airport', 'longitude', -0.461941], ['node', ['PASSENGERS'], 'Airport', 'latitude', 51.4706], ['node', ['PASSENGERS'], 'Airport', 'runwaysWidth', 164.0], ['node', ['PASSENGERS'], 'Airport', 'airport_id', 507], ['node', ['OF'], 'City', 'health_care_index', 71.2], ['node', ['OF'], 'City', 'countrycode', 'au'], ['node', ['OF'], 'City', 'accent_name', 'Perth'], ['node', ['OF'], 'City', 'location', [115.861397, -31.952240000000003]], ['node', ['OF'], 'City', 'climate_index', 95.7], ['node', ['OF'], 'City', 'purchasing_power_index', 140.8], ['node', ['OF'], 'City', 'traffic_commute_time_index', 42.4], ['node', ['OF'], 'City', 'cost_of_living_index', 69.6], ['node', ['OF'], 'City', 'country', 'Australia'], ['node', ['OF'], 'City', 'pollution_index', 27.7], ['node', ['OF'], 'City', 'quality_of_life_index', 193.8], ['node', ['OF'], 'City', 'rank', 14], ['node', ['OF'], 'City', 'safety_index', 58.3], ['node', ['OF'], 'City', 'name', 'perth'], ['node', ['OF'], 'City', 'longitude', 115.861397], ['node', ['OF'], 'City', 'property_price_to_income_ratio', 5.5], ['node', ['OF'], 'City', 'latitude', -31.952240000000003], ['node', ['OF'], 'City', 'population', 1446715.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'location', [28.246, -26.1392]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5558.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12825.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'South Africa'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JNB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FAOR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'OR Tambo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20930.74996990909], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 28.246], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -26.1392], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 198.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 813], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'location', [57.683601, -20.430201]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 186.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11056.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Plaisance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MRU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FIMP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sir Seewoosagur Ramgoolam International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16812384.586103693], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 57.683601], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.430201], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 893], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Pacific/Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'location', [174.792007446, -37.0080986023]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 23.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'Z'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11061.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'New Zealand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AKL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '12'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'NZAA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Auckland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15904.7306655827], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 174.792007446], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.0080986023], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2006], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [55.3643989563, 25.252799987800003]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 62.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14432.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United Arab Emirates'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OMDB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dubai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6038.403573841609], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 55.3643989563], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.252799987800003], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2188], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Hong_Kong'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.915001, 22.308901]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 28.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12467.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VHHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hong Kong International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 24370.906942983245], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.915001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.308901], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3077], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [98.316902, 8.1132]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 82.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Phuket'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTSP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phuket International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 32003.874082687587], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 98.316902], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 8.1132], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3179], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.0510025024414, 5.9372100830078125]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12402.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kota Kinabalu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WBKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kota Kinabalu International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4818.2885301308415], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.0510025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 5.9372100830078125], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3269], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.65599823, -6.1255698204]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 34.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11221.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CGK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WIII'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Soekarno-Hatta International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19952.873327230303], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.65599823], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -6.1255698204], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3275], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [101.70999908446998, 2.745579957962]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 69.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13270.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kuala Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KUL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WMKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kuala Lumpur International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 46888.395782021675], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 101.70999908446998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 2.745579957962], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3304], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'location', [103.994003, 1.35019]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 22.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SIN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Singapore Changi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16640.33491669021], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 103.994003], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 1.35019], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3316], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [133.90199279785156, -23.806699752807614]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1789.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5858.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Alice Springs'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ASP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Alice Springs Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12301.660108031643], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 133.90199279785156], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.806699752807614], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [153.11700439453125, -27.38419914245605]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9361.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBBN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Brisbane International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13315.566428894339], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 153.11700439453125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -27.38419914245605], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.66666666666666], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3320], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [145.755004883, -16.885799408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6762.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairns'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CNS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBCS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairns International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4315.184422636413], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 145.755004883], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -16.885799408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3322], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Hobart'], ['node', ['ROUTE_TO'], 'Airport', 'location', [144.843002, -37.673302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 434.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9749.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Melbourne'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YMML'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Melbourne International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18906.748936221637], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 144.843002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.673302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3339], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'location', [138.53100600000002, -34.945]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 20.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7795.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ADL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPAD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Adelaide International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6433.945978837181], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 138.53100600000002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.945], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3341], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.773002625, -20.712200164799995]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 29.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7480.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Karratha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KTA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Karratha Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8104.177830659545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.773002625], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.712200164799995], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3345], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.461997986, -30.7894001007]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1203.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5249.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kalgoorlie'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KGI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kalgoorlie Boulder Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.461997986], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -30.7894001007], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3346], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [128.707992554, -15.7781000137]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 145.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kununurra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KNX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kununurra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 3872.4122806130536], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 128.707992554], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -15.7781000137], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3347], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.088996887, -22.2355995178]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 19.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9997.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Learmonth'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPLM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Learmonth Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1705.976705317044], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.088996887], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -22.2355995178], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3348], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [118.625999451, -20.3777999878]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5741.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Port Hedland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PHE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPPD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Port Hedland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7483.170510964459], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 118.625999451], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.3777999878], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3349], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Christmas'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.69000244140624, -10.450599670410156]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 916.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'XCH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPXM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Christmas Island Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.69000244140624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -10.450599670410156], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3353], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [149.19500732421875, -35.30690002441406]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1886.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8140.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Canberra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CBR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSCB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Canberra International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6494.989912504808], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 149.19500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -35.30690002441406], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 123.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3355], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [151.177001953125, -33.94609832763672]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 21.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9766.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SYD'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSSY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sydney Kingsford Smith International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9780.015780688975], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 151.177001953125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.94609832763672], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3361], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.16699981689, -8.7481698989868]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 14.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9790.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denpasar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DPS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ngurah Rai (Bali) International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12218.671924212083], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.16699981689], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.7481698989868], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3940], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [130.87699890136722, -12.41469955444336]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 103.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8002.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DRW'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPDN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Darwin International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6454.0079836886625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 130.87699890136722], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -12.41469955444336], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3999], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [122.23200225830078, -17.944700241088867]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 56.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8064.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Broome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBRM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Broome International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1413.2920249796625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 122.23200225830078], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.944700241088867], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 4319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.80899810791016, -34.94329833984375]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 233.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4751.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Albany'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ALH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YABA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Albany Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8444.428815978767], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.80899810791016], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.94329833984375], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6235], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.822998, -33.684399]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 470.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4393.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Esperance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'EPR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YESP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Esperance Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20324.669691320036], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.822998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.684399], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 78.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6266], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.707001, -28.796101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 121.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4520.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Geraldton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GET'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YGEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Geraldton Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9214.850207397747], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.707001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -28.796101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 88.66666666666667], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6268], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [119.803001404, -23.4178009033]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1724.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6798.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Newman'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ZNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNWN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Newman Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17766397.24623855], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 119.803001404], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.4178009033], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6310], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.745002747, -23.1711006165]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1406.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6995.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Paraburdoo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPBO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paraburdoo Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8536.853726239466], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.745002747], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.1711006165], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6314], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [120.208000183, -33.7971992493]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4719.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ravensthorpe'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'RVT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNRV'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ravensthorpe Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28126.50928224029], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 120.208000183], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.7971992493], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.276675, -8.757322]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 319.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10826.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Praya'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LOP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lombok International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5825.787746988739], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.276675], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.757322], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 8401], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.401596069, -33.688423156700004]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 55.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8071.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brusselton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BQB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBLN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Busselton Regional Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.401596069], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.688423156700004], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9090], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [123.82800293, -17.5813999176]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 300.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10003.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Derby'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DCN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YCIN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'RAAF Base Curtin'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9823191.307539554], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 123.82800293], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.5813999176], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9905], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [51.608056, 25.273056]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14928.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Qatar'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Doha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DOH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OTHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hamad International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7672.619050740817], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 51.608056], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.273056], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 11051], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_passengers', 141724], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_seats', 170516], ['edge', ['PASSENGERS'], 'PASSENGERS', 'year', 2019], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.2025362631243], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.6812558523761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 508.3855045687054], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.4195670320667], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.9336857336278], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.2602056814085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.0011820925939], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.56280175906863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.7159531338469], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.5185846231632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.0121582810918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.83590604039534], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.5730050875931], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.8491382234394], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.5445402122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.5749494563072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.0590134569188], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.9649762790181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.0978963191487], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.1547049279857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.0401004744097], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.164223519384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.3973415137738], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.527206217356], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 529.331727651091], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.0067937752065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.1128873353298], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 503.022373482305], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9100121990385], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.8438264333045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.8284140757384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.1507764396483], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0272525066715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.3832727304101], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.3853286503956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7918180563407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.7277474100887], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.7825063331557], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.6767583696729], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.87710007569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 599.258508028423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 511.9551573456317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.757888902113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8933704052415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.6972325013244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.4522900590486], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 543.2842554063412], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4762011490865], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.65236227784754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.94371412093443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.7820825772044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.38149947257597], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.5785521775655], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9931946073007], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '763 332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.30833351976105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 422.6451817359142], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H 332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 554.0060133377134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 426.6662319432057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 511.85298147531245], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.1555991084923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4372219567109], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 570.6928092937061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.5567371540335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.90146483345325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.302208945234], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.1743450662014], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.3040499977301], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.92384963415446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.1525332976813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.81568949199465], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 522.0320489949096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.4980270116798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.4562370175372], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.08500886824663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2658369157759], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.26118464194604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.49828784727066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.0114099329785], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2285907391058], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.07926429842854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9495164918967], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.32065224981454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 526.1890724701276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9643636553158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 491.79350327381326], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8114645776544], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.5502812409808], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.300690686633], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.0654228054552], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.37743438802613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 644.1322969336072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.401987349898], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0879923327558], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.2246147253314], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 560.1843120692553], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.8236016915419], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5279712736085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8338597725095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3783253674443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.8907645828957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.0473870510434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.27340288710286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.84193195240096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.1584360944793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 600.4976321939617], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.52723380292883], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.576665719835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.12928192915075], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.3450509823031], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.62702904718844], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 623.2478869923331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.23676349117994], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.0738477550975], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.17297155461915], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.24125816404575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9878080264196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 553.1233497615187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7612754438496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.5764996859718], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.3922640167573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.8867020874468], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.769936063906], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.7557861418393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 490.80451159920466], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.812560045748], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.451051921253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.2724915064517], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.3971759221196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.5673321867221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.4832090246027], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 588.7868532252173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 557.574077976966], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.6850274492022], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.9755905674709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 569.83923728527], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 423.09541058313334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 609.6896746165189], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 527.7880286270662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.2803588207797], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.65906034752857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.90853756326095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 449.5837654569756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.6641452108297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.7689328926849], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 498.770441590863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.50514554959426], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.8624654842458], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.9097897715537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.9544570378813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.4111489817221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5212185987317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.5262062926587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.9372505865084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.4913301585447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.3211289617824], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.1247113251569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 524.8573332645819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.6387263730197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 617.9044601153835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 491.84039030061854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.4450859009229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.3560115012395], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.878776081061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.7492124375674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.52622439576373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.69408678252256], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.5798952957598], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.79117929486375], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.7932988360084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.792872338254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 647.6118748516647], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.5680220001173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.63697860116866], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9731646454644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.8237473570492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.8108334656422], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.1638308677572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.6013239892299], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.5707317247616], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 443.1146897398341], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.9220369244747], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6300487111251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.4271990225716], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 500.4818137220986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 527.3092645496879], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.1879776081525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4353057347079], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.87994878804193], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.3949377382753], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.4637803744531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.9473396540226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.8560822813565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 630.7406028012334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.99202148064364], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.0356830700694], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.32594667452827], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.3746657423881], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.67122637563216], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4508123787871], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.6592312184615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.8585099800715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.9736893015839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 637.9122175107984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.1955137797173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 494.7199387952659], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.1828149796918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 601.1289947975267], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.6191771001185], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.3381354583017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 429.20045912894676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.8371536532313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.9270167548373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.9471248494835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.9973054867768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.1462500003859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.72112561180666], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 593.530389915528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.1838541142181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.1934891188854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.4917751343677], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.9697232330274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.2333596548143], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.7673486386564], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.3416397129491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 541.3561451250066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.62311676302494], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.8925710791717], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.1620814376962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.1031905125329], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.8432710498705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 597.9603451276024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.6430527086839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.3471628270505], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.8317489600514], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.839050083348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.5282626287961], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.43238971765805], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.9789349409814], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.2225019092525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.6101465041956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.824918232652], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.4688309145857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.7877852003875], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 482.6542954602548], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 595.0953007820791], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.49141742587625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 558.8870055818687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.0734509395793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.0081276901934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.5329090386708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.3652686202531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.0375817854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 518.6391993179819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 542.0690151842002], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.6301054572286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6283545902703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 533.2235287056707], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 466.030544359565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.8657588521703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.77778397141594], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.4906821300837], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.3969869444828], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.1938130322338], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.995998341848], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.3008050449005], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 535.9263274096863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.6137983573092], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.3386484551024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 528.6181134761199], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.5404899634831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.7783964998134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.72294112061587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.3717443991761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.49037803741504], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.1276304066477], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.0887682739982], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.4324990659869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9704766909796], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.5835883587526], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.1201700574769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.5119849438112], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.505930774511], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.77830300641017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.09830838168074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3751516961829], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.3759350249068], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.8567063908843], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.8473922263536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.0541165301304], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.6980524012438], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0042480933749], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 476.22996376368997], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.388406854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.9420178487572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.72697344472533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.0384597579654], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4331532452838], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.5098129536639], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.9268983185507], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9359781705669], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.4849185399664], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4731835234663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.902686328858], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 508.43807633236105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 515.7254399167086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.999392475016], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.1677136578253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 439.65591364088266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.194782498046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0862397400867], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.60838425139934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.24921735529244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.81250379760434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.87066150264604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.8094003498974], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.55501106888175], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 542.9622092091613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.17469870172033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.40910580320497], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 510.7352947198684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2855419123816], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.8726076692513], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0840283113087], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.9480229133085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Oceania'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 65.5], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 3.68], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.27], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.93], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 1392680589329.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 1.6], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Canberra'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 375908.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Sydney'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'AUS'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 47.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 61], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.0], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 53.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 58000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 16.3], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.74], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 133.775136], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'AUD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 21844756.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 119.8], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 25766605.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 3.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 7741220.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 13.59], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '036'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 12.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 6.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 9.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 113.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', -25.274398], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Australia and New Zealand'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.7], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 48.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 19.6]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: dst:N,icao:YPPH,longitude:115.96700286865234\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: source,name,latitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('YPPH', ['timezone', 'name', 'latitude'], ['runways', 'runwaysLength', 'runwaysWidth'], {'difficultyProp': 'easyhard', 'entityDiff:': 'easy', 'context': [['node', ['PASSENGERS'], 'Airport', 'tz_database_timezone', 'Europe/London'], ['node', ['PASSENGERS'], 'Airport', 'commercial_flights', 474576], ['node', ['PASSENGERS'], 'Airport', 'location', [-0.461941, 51.4706]], ['node', ['PASSENGERS'], 'Airport', 'altitude', 83.0], ['node', ['PASSENGERS'], 'Airport', 'dst', 'E'], ['node', ['PASSENGERS'], 'Airport', 'type', 'airport'], ['node', ['PASSENGERS'], 'Airport', 'runwaysLength', 12400.0], ['node', ['PASSENGERS'], 'Airport', 'country', 'United Kingdom'], ['node', ['PASSENGERS'], 'Airport', 'city', 'London'], ['node', ['PASSENGERS'], 'Airport', 'iata', 'LHR'], ['node', ['PASSENGERS'], 'Airport', 'timezone', '0'], ['node', ['PASSENGERS'], 'Airport', 'source', 'OurAirports'], ['node', ['PASSENGERS'], 'Airport', 'runways', 2.0], ['node', ['PASSENGERS'], 'Airport', 'icao', 'EGLL'], ['node', ['PASSENGERS'], 'Airport', 'name', 'London Heathrow Airport'], ['node', ['PASSENGERS'], 'Airport', 'distCity', 25979.20729323409], ['node', ['PASSENGERS'], 'Airport', 'longitude', -0.461941], ['node', ['PASSENGERS'], 'Airport', 'latitude', 51.4706], ['node', ['PASSENGERS'], 'Airport', 'runwaysWidth', 164.0], ['node', ['PASSENGERS'], 'Airport', 'airport_id', 507], ['node', ['OF'], 'City', 'health_care_index', 71.2], ['node', ['OF'], 'City', 'countrycode', 'au'], ['node', ['OF'], 'City', 'accent_name', 'Perth'], ['node', ['OF'], 'City', 'location', [115.861397, -31.952240000000003]], ['node', ['OF'], 'City', 'climate_index', 95.7], ['node', ['OF'], 'City', 'purchasing_power_index', 140.8], ['node', ['OF'], 'City', 'traffic_commute_time_index', 42.4], ['node', ['OF'], 'City', 'cost_of_living_index', 69.6], ['node', ['OF'], 'City', 'country', 'Australia'], ['node', ['OF'], 'City', 'pollution_index', 27.7], ['node', ['OF'], 'City', 'quality_of_life_index', 193.8], ['node', ['OF'], 'City', 'rank', 14], ['node', ['OF'], 'City', 'safety_index', 58.3], ['node', ['OF'], 'City', 'name', 'perth'], ['node', ['OF'], 'City', 'longitude', 115.861397], ['node', ['OF'], 'City', 'property_price_to_income_ratio', 5.5], ['node', ['OF'], 'City', 'latitude', -31.952240000000003], ['node', ['OF'], 'City', 'population', 1446715.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'location', [28.246, -26.1392]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5558.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12825.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'South Africa'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JNB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FAOR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'OR Tambo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20930.74996990909], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 28.246], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -26.1392], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 198.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 813], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'location', [57.683601, -20.430201]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 186.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11056.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Plaisance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MRU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FIMP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sir Seewoosagur Ramgoolam International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16812384.586103693], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 57.683601], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.430201], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 893], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Pacific/Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'location', [174.792007446, -37.0080986023]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 23.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'Z'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11061.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'New Zealand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AKL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '12'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'NZAA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Auckland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15904.7306655827], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 174.792007446], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.0080986023], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2006], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [55.3643989563, 25.252799987800003]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 62.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14432.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United Arab Emirates'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OMDB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dubai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6038.403573841609], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 55.3643989563], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.252799987800003], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2188], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Hong_Kong'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.915001, 22.308901]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 28.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12467.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VHHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hong Kong International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 24370.906942983245], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.915001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.308901], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3077], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [98.316902, 8.1132]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 82.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Phuket'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTSP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phuket International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 32003.874082687587], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 98.316902], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 8.1132], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3179], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.0510025024414, 5.9372100830078125]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12402.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kota Kinabalu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WBKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kota Kinabalu International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4818.2885301308415], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.0510025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 5.9372100830078125], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3269], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.65599823, -6.1255698204]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 34.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11221.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CGK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WIII'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Soekarno-Hatta International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19952.873327230303], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.65599823], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -6.1255698204], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3275], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [101.70999908446998, 2.745579957962]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 69.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13270.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kuala Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KUL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WMKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kuala Lumpur International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 46888.395782021675], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 101.70999908446998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 2.745579957962], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3304], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'location', [103.994003, 1.35019]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 22.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SIN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Singapore Changi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16640.33491669021], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 103.994003], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 1.35019], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3316], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [133.90199279785156, -23.806699752807614]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1789.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5858.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Alice Springs'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ASP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Alice Springs Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12301.660108031643], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 133.90199279785156], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.806699752807614], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [153.11700439453125, -27.38419914245605]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9361.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBBN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Brisbane International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13315.566428894339], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 153.11700439453125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -27.38419914245605], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.66666666666666], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3320], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [145.755004883, -16.885799408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6762.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairns'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CNS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBCS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairns International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4315.184422636413], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 145.755004883], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -16.885799408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3322], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Hobart'], ['node', ['ROUTE_TO'], 'Airport', 'location', [144.843002, -37.673302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 434.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9749.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Melbourne'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YMML'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Melbourne International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18906.748936221637], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 144.843002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.673302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3339], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'location', [138.53100600000002, -34.945]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 20.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7795.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ADL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPAD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Adelaide International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6433.945978837181], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 138.53100600000002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.945], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3341], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.773002625, -20.712200164799995]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 29.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7480.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Karratha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KTA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Karratha Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8104.177830659545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.773002625], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.712200164799995], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3345], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.461997986, -30.7894001007]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1203.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5249.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kalgoorlie'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KGI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kalgoorlie Boulder Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.461997986], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -30.7894001007], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3346], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [128.707992554, -15.7781000137]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 145.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kununurra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KNX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kununurra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 3872.4122806130536], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 128.707992554], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -15.7781000137], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3347], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.088996887, -22.2355995178]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 19.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9997.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Learmonth'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPLM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Learmonth Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1705.976705317044], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.088996887], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -22.2355995178], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3348], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [118.625999451, -20.3777999878]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5741.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Port Hedland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PHE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPPD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Port Hedland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7483.170510964459], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 118.625999451], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.3777999878], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3349], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Christmas'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.69000244140624, -10.450599670410156]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 916.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'XCH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPXM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Christmas Island Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.69000244140624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -10.450599670410156], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3353], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [149.19500732421875, -35.30690002441406]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1886.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8140.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Canberra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CBR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSCB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Canberra International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6494.989912504808], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 149.19500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -35.30690002441406], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 123.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3355], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [151.177001953125, -33.94609832763672]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 21.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9766.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SYD'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSSY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sydney Kingsford Smith International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9780.015780688975], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 151.177001953125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.94609832763672], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3361], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.16699981689, -8.7481698989868]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 14.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9790.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denpasar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DPS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ngurah Rai (Bali) International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12218.671924212083], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.16699981689], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.7481698989868], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3940], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [130.87699890136722, -12.41469955444336]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 103.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8002.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DRW'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPDN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Darwin International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6454.0079836886625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 130.87699890136722], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -12.41469955444336], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3999], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [122.23200225830078, -17.944700241088867]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 56.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8064.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Broome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBRM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Broome International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1413.2920249796625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 122.23200225830078], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.944700241088867], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 4319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.80899810791016, -34.94329833984375]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 233.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4751.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Albany'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ALH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YABA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Albany Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8444.428815978767], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.80899810791016], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.94329833984375], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6235], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.822998, -33.684399]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 470.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4393.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Esperance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'EPR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YESP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Esperance Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20324.669691320036], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.822998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.684399], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 78.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6266], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.707001, -28.796101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 121.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4520.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Geraldton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GET'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YGEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Geraldton Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9214.850207397747], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.707001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -28.796101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 88.66666666666667], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6268], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [119.803001404, -23.4178009033]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1724.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6798.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Newman'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ZNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNWN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Newman Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17766397.24623855], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 119.803001404], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.4178009033], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6310], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.745002747, -23.1711006165]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1406.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6995.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Paraburdoo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPBO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paraburdoo Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8536.853726239466], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.745002747], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.1711006165], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6314], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [120.208000183, -33.7971992493]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4719.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ravensthorpe'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'RVT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNRV'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ravensthorpe Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28126.50928224029], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 120.208000183], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.7971992493], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.276675, -8.757322]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 319.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10826.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Praya'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LOP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lombok International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5825.787746988739], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.276675], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.757322], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 8401], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.401596069, -33.688423156700004]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 55.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8071.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brusselton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BQB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBLN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Busselton Regional Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.401596069], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.688423156700004], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9090], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [123.82800293, -17.5813999176]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 300.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10003.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Derby'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DCN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YCIN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'RAAF Base Curtin'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9823191.307539554], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 123.82800293], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.5813999176], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9905], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [51.608056, 25.273056]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14928.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Qatar'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Doha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DOH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OTHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hamad International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7672.619050740817], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 51.608056], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.273056], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 11051], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_passengers', 141724], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_seats', 170516], ['edge', ['PASSENGERS'], 'PASSENGERS', 'year', 2019], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.2025362631243], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.6812558523761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 508.3855045687054], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.4195670320667], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.9336857336278], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.2602056814085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.0011820925939], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.56280175906863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.7159531338469], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.5185846231632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.0121582810918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.83590604039534], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.5730050875931], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.8491382234394], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.5445402122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.5749494563072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.0590134569188], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.9649762790181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.0978963191487], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.1547049279857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.0401004744097], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.164223519384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.3973415137738], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.527206217356], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 529.331727651091], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.0067937752065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.1128873353298], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 503.022373482305], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9100121990385], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.8438264333045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.8284140757384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.1507764396483], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0272525066715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.3832727304101], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.3853286503956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7918180563407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.7277474100887], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.7825063331557], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.6767583696729], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.87710007569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 599.258508028423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 511.9551573456317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.757888902113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8933704052415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.6972325013244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.4522900590486], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 543.2842554063412], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4762011490865], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.65236227784754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.94371412093443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.7820825772044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.38149947257597], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.5785521775655], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9931946073007], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '763 332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.30833351976105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 422.6451817359142], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H 332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 554.0060133377134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 426.6662319432057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 511.85298147531245], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.1555991084923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4372219567109], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 570.6928092937061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.5567371540335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.90146483345325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.302208945234], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.1743450662014], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.3040499977301], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.92384963415446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.1525332976813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.81568949199465], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 522.0320489949096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.4980270116798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.4562370175372], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.08500886824663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2658369157759], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.26118464194604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.49828784727066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.0114099329785], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2285907391058], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.07926429842854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9495164918967], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.32065224981454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 526.1890724701276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9643636553158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 491.79350327381326], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8114645776544], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.5502812409808], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.300690686633], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.0654228054552], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.37743438802613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 644.1322969336072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.401987349898], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0879923327558], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.2246147253314], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 560.1843120692553], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.8236016915419], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5279712736085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8338597725095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3783253674443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.8907645828957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.0473870510434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.27340288710286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.84193195240096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.1584360944793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 600.4976321939617], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.52723380292883], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.576665719835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.12928192915075], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.3450509823031], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.62702904718844], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 623.2478869923331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.23676349117994], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.0738477550975], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.17297155461915], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.24125816404575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9878080264196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 553.1233497615187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7612754438496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.5764996859718], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.3922640167573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.8867020874468], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.769936063906], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.7557861418393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 490.80451159920466], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.812560045748], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.451051921253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.2724915064517], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.3971759221196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.5673321867221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.4832090246027], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 588.7868532252173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 557.574077976966], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.6850274492022], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.9755905674709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 569.83923728527], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 423.09541058313334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 609.6896746165189], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 527.7880286270662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.2803588207797], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.65906034752857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.90853756326095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 449.5837654569756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.6641452108297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.7689328926849], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 498.770441590863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.50514554959426], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.8624654842458], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.9097897715537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.9544570378813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.4111489817221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5212185987317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.5262062926587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.9372505865084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.4913301585447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.3211289617824], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.1247113251569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 524.8573332645819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.6387263730197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 617.9044601153835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 491.84039030061854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.4450859009229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.3560115012395], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.878776081061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.7492124375674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.52622439576373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.69408678252256], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.5798952957598], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.79117929486375], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.7932988360084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.792872338254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 647.6118748516647], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.5680220001173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.63697860116866], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9731646454644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.8237473570492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.8108334656422], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.1638308677572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.6013239892299], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.5707317247616], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 443.1146897398341], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.9220369244747], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6300487111251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.4271990225716], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 500.4818137220986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 527.3092645496879], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.1879776081525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4353057347079], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.87994878804193], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.3949377382753], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.4637803744531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.9473396540226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.8560822813565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 630.7406028012334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.99202148064364], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.0356830700694], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.32594667452827], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.3746657423881], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.67122637563216], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4508123787871], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.6592312184615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.8585099800715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.9736893015839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 637.9122175107984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.1955137797173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 494.7199387952659], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.1828149796918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 601.1289947975267], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.6191771001185], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.3381354583017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 429.20045912894676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.8371536532313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.9270167548373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.9471248494835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.9973054867768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.1462500003859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.72112561180666], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 593.530389915528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.1838541142181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.1934891188854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.4917751343677], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.9697232330274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.2333596548143], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.7673486386564], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.3416397129491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 541.3561451250066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.62311676302494], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.8925710791717], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.1620814376962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.1031905125329], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.8432710498705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 597.9603451276024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.6430527086839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.3471628270505], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.8317489600514], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.839050083348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.5282626287961], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.43238971765805], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.9789349409814], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.2225019092525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.6101465041956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.824918232652], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.4688309145857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.7877852003875], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 482.6542954602548], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 595.0953007820791], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.49141742587625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 558.8870055818687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.0734509395793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.0081276901934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.5329090386708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.3652686202531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.0375817854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 518.6391993179819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 542.0690151842002], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.6301054572286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6283545902703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 533.2235287056707], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 466.030544359565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.8657588521703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.77778397141594], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.4906821300837], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.3969869444828], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.1938130322338], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.995998341848], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.3008050449005], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 535.9263274096863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.6137983573092], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.3386484551024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 528.6181134761199], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.5404899634831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.7783964998134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.72294112061587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.3717443991761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.49037803741504], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.1276304066477], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.0887682739982], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.4324990659869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9704766909796], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.5835883587526], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.1201700574769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.5119849438112], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.505930774511], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.77830300641017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.09830838168074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3751516961829], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.3759350249068], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.8567063908843], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.8473922263536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.0541165301304], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.6980524012438], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0042480933749], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 476.22996376368997], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.388406854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.9420178487572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.72697344472533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.0384597579654], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4331532452838], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.5098129536639], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.9268983185507], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9359781705669], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.4849185399664], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4731835234663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.902686328858], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 508.43807633236105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 515.7254399167086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.999392475016], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.1677136578253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 439.65591364088266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.194782498046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0862397400867], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.60838425139934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.24921735529244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.81250379760434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.87066150264604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.8094003498974], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.55501106888175], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 542.9622092091613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.17469870172033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.40910580320497], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 510.7352947198684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2855419123816], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.8726076692513], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0840283113087], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.9480229133085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Oceania'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 65.5], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 3.68], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.27], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.93], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 1392680589329.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 1.6], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Canberra'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 375908.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Sydney'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'AUS'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 47.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 61], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.0], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 53.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 58000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 16.3], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.74], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 133.775136], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'AUD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 21844756.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 119.8], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 25766605.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 3.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 7741220.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 13.59], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '036'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 12.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 6.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 9.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 113.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', -25.274398], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Australia and New Zealand'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.7], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 48.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 19.6]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:9197.5,runways:2.0,runwaysWidth:148.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: timezone,name,latitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('YPPH', ['runwaysLength', 'runways', 'iata'], ['distCity', 'type', 'icao'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'easy', 'context': [['node', ['PASSENGERS'], 'Airport', 'tz_database_timezone', 'Europe/London'], ['node', ['PASSENGERS'], 'Airport', 'commercial_flights', 474576], ['node', ['PASSENGERS'], 'Airport', 'location', [-0.461941, 51.4706]], ['node', ['PASSENGERS'], 'Airport', 'altitude', 83.0], ['node', ['PASSENGERS'], 'Airport', 'dst', 'E'], ['node', ['PASSENGERS'], 'Airport', 'type', 'airport'], ['node', ['PASSENGERS'], 'Airport', 'runwaysLength', 12400.0], ['node', ['PASSENGERS'], 'Airport', 'country', 'United Kingdom'], ['node', ['PASSENGERS'], 'Airport', 'city', 'London'], ['node', ['PASSENGERS'], 'Airport', 'iata', 'LHR'], ['node', ['PASSENGERS'], 'Airport', 'timezone', '0'], ['node', ['PASSENGERS'], 'Airport', 'source', 'OurAirports'], ['node', ['PASSENGERS'], 'Airport', 'runways', 2.0], ['node', ['PASSENGERS'], 'Airport', 'icao', 'EGLL'], ['node', ['PASSENGERS'], 'Airport', 'name', 'London Heathrow Airport'], ['node', ['PASSENGERS'], 'Airport', 'distCity', 25979.20729323409], ['node', ['PASSENGERS'], 'Airport', 'longitude', -0.461941], ['node', ['PASSENGERS'], 'Airport', 'latitude', 51.4706], ['node', ['PASSENGERS'], 'Airport', 'runwaysWidth', 164.0], ['node', ['PASSENGERS'], 'Airport', 'airport_id', 507], ['node', ['OF'], 'City', 'health_care_index', 71.2], ['node', ['OF'], 'City', 'countrycode', 'au'], ['node', ['OF'], 'City', 'accent_name', 'Perth'], ['node', ['OF'], 'City', 'location', [115.861397, -31.952240000000003]], ['node', ['OF'], 'City', 'climate_index', 95.7], ['node', ['OF'], 'City', 'purchasing_power_index', 140.8], ['node', ['OF'], 'City', 'traffic_commute_time_index', 42.4], ['node', ['OF'], 'City', 'cost_of_living_index', 69.6], ['node', ['OF'], 'City', 'country', 'Australia'], ['node', ['OF'], 'City', 'pollution_index', 27.7], ['node', ['OF'], 'City', 'quality_of_life_index', 193.8], ['node', ['OF'], 'City', 'rank', 14], ['node', ['OF'], 'City', 'safety_index', 58.3], ['node', ['OF'], 'City', 'name', 'perth'], ['node', ['OF'], 'City', 'longitude', 115.861397], ['node', ['OF'], 'City', 'property_price_to_income_ratio', 5.5], ['node', ['OF'], 'City', 'latitude', -31.952240000000003], ['node', ['OF'], 'City', 'population', 1446715.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'location', [28.246, -26.1392]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5558.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12825.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'South Africa'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JNB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FAOR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'OR Tambo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20930.74996990909], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 28.246], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -26.1392], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 198.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 813], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'location', [57.683601, -20.430201]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 186.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11056.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Plaisance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MRU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FIMP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sir Seewoosagur Ramgoolam International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16812384.586103693], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 57.683601], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.430201], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 893], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Pacific/Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'location', [174.792007446, -37.0080986023]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 23.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'Z'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11061.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'New Zealand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AKL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '12'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'NZAA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Auckland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15904.7306655827], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 174.792007446], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.0080986023], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2006], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [55.3643989563, 25.252799987800003]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 62.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14432.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United Arab Emirates'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OMDB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dubai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6038.403573841609], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 55.3643989563], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.252799987800003], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2188], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Hong_Kong'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.915001, 22.308901]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 28.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12467.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VHHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hong Kong International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 24370.906942983245], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.915001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.308901], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3077], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [98.316902, 8.1132]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 82.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Phuket'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTSP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phuket International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 32003.874082687587], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 98.316902], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 8.1132], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3179], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.0510025024414, 5.9372100830078125]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12402.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kota Kinabalu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WBKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kota Kinabalu International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4818.2885301308415], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.0510025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 5.9372100830078125], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3269], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.65599823, -6.1255698204]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 34.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11221.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CGK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WIII'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Soekarno-Hatta International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19952.873327230303], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.65599823], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -6.1255698204], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3275], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [101.70999908446998, 2.745579957962]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 69.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13270.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kuala Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KUL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WMKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kuala Lumpur International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 46888.395782021675], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 101.70999908446998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 2.745579957962], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3304], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'location', [103.994003, 1.35019]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 22.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SIN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Singapore Changi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16640.33491669021], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 103.994003], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 1.35019], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3316], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [133.90199279785156, -23.806699752807614]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1789.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5858.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Alice Springs'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ASP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Alice Springs Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12301.660108031643], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 133.90199279785156], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.806699752807614], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [153.11700439453125, -27.38419914245605]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9361.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBBN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Brisbane International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13315.566428894339], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 153.11700439453125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -27.38419914245605], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.66666666666666], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3320], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [145.755004883, -16.885799408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6762.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairns'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CNS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBCS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairns International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4315.184422636413], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 145.755004883], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -16.885799408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3322], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Hobart'], ['node', ['ROUTE_TO'], 'Airport', 'location', [144.843002, -37.673302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 434.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9749.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Melbourne'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YMML'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Melbourne International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18906.748936221637], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 144.843002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.673302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3339], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'location', [138.53100600000002, -34.945]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 20.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7795.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ADL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPAD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Adelaide International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6433.945978837181], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 138.53100600000002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.945], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3341], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.773002625, -20.712200164799995]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 29.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7480.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Karratha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KTA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Karratha Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8104.177830659545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.773002625], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.712200164799995], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3345], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.461997986, -30.7894001007]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1203.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5249.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kalgoorlie'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KGI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kalgoorlie Boulder Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.461997986], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -30.7894001007], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3346], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [128.707992554, -15.7781000137]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 145.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kununurra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KNX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kununurra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 3872.4122806130536], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 128.707992554], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -15.7781000137], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3347], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.088996887, -22.2355995178]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 19.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9997.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Learmonth'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPLM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Learmonth Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1705.976705317044], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.088996887], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -22.2355995178], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3348], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [118.625999451, -20.3777999878]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5741.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Port Hedland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PHE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPPD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Port Hedland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7483.170510964459], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 118.625999451], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.3777999878], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3349], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Christmas'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.69000244140624, -10.450599670410156]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 916.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'XCH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPXM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Christmas Island Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.69000244140624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -10.450599670410156], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3353], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [149.19500732421875, -35.30690002441406]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1886.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8140.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Canberra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CBR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSCB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Canberra International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6494.989912504808], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 149.19500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -35.30690002441406], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 123.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3355], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [151.177001953125, -33.94609832763672]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 21.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9766.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SYD'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSSY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sydney Kingsford Smith International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9780.015780688975], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 151.177001953125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.94609832763672], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3361], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.16699981689, -8.7481698989868]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 14.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9790.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denpasar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DPS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ngurah Rai (Bali) International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12218.671924212083], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.16699981689], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.7481698989868], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3940], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [130.87699890136722, -12.41469955444336]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 103.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8002.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DRW'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPDN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Darwin International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6454.0079836886625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 130.87699890136722], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -12.41469955444336], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3999], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [122.23200225830078, -17.944700241088867]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 56.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8064.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Broome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBRM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Broome International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1413.2920249796625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 122.23200225830078], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.944700241088867], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 4319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.80899810791016, -34.94329833984375]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 233.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4751.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Albany'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ALH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YABA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Albany Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8444.428815978767], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.80899810791016], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.94329833984375], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6235], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.822998, -33.684399]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 470.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4393.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Esperance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'EPR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YESP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Esperance Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20324.669691320036], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.822998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.684399], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 78.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6266], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.707001, -28.796101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 121.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4520.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Geraldton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GET'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YGEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Geraldton Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9214.850207397747], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.707001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -28.796101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 88.66666666666667], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6268], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [119.803001404, -23.4178009033]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1724.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6798.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Newman'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ZNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNWN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Newman Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17766397.24623855], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 119.803001404], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.4178009033], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6310], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.745002747, -23.1711006165]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1406.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6995.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Paraburdoo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPBO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paraburdoo Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8536.853726239466], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.745002747], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.1711006165], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6314], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [120.208000183, -33.7971992493]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4719.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ravensthorpe'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'RVT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNRV'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ravensthorpe Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28126.50928224029], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 120.208000183], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.7971992493], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.276675, -8.757322]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 319.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10826.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Praya'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LOP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lombok International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5825.787746988739], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.276675], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.757322], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 8401], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.401596069, -33.688423156700004]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 55.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8071.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brusselton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BQB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBLN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Busselton Regional Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.401596069], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.688423156700004], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9090], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [123.82800293, -17.5813999176]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 300.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10003.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Derby'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DCN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YCIN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'RAAF Base Curtin'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9823191.307539554], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 123.82800293], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.5813999176], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9905], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [51.608056, 25.273056]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14928.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Qatar'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Doha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DOH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OTHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hamad International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7672.619050740817], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 51.608056], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.273056], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 11051], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_passengers', 141724], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_seats', 170516], ['edge', ['PASSENGERS'], 'PASSENGERS', 'year', 2019], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.2025362631243], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.6812558523761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 508.3855045687054], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.4195670320667], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.9336857336278], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.2602056814085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.0011820925939], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.56280175906863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.7159531338469], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.5185846231632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.0121582810918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.83590604039534], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.5730050875931], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.8491382234394], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.5445402122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.5749494563072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.0590134569188], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.9649762790181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.0978963191487], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.1547049279857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.0401004744097], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.164223519384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.3973415137738], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.527206217356], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 529.331727651091], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.0067937752065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.1128873353298], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 503.022373482305], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9100121990385], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.8438264333045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.8284140757384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.1507764396483], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0272525066715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.3832727304101], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.3853286503956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7918180563407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.7277474100887], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.7825063331557], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.6767583696729], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.87710007569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 599.258508028423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 511.9551573456317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.757888902113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8933704052415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.6972325013244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.4522900590486], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 543.2842554063412], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4762011490865], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.65236227784754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.94371412093443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.7820825772044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.38149947257597], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.5785521775655], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9931946073007], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '763 332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.30833351976105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 422.6451817359142], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H 332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 554.0060133377134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 426.6662319432057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 511.85298147531245], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.1555991084923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4372219567109], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 570.6928092937061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.5567371540335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.90146483345325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.302208945234], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.1743450662014], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.3040499977301], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.92384963415446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.1525332976813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.81568949199465], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 522.0320489949096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.4980270116798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.4562370175372], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.08500886824663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2658369157759], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.26118464194604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.49828784727066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.0114099329785], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2285907391058], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.07926429842854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9495164918967], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.32065224981454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 526.1890724701276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9643636553158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 491.79350327381326], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8114645776544], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.5502812409808], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.300690686633], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.0654228054552], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.37743438802613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 644.1322969336072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.401987349898], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0879923327558], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.2246147253314], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 560.1843120692553], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.8236016915419], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5279712736085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8338597725095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3783253674443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.8907645828957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.0473870510434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.27340288710286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.84193195240096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.1584360944793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 600.4976321939617], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.52723380292883], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.576665719835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.12928192915075], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.3450509823031], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.62702904718844], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 623.2478869923331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.23676349117994], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.0738477550975], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.17297155461915], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.24125816404575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9878080264196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 553.1233497615187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7612754438496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.5764996859718], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.3922640167573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.8867020874468], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.769936063906], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.7557861418393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 490.80451159920466], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.812560045748], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.451051921253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.2724915064517], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.3971759221196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.5673321867221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.4832090246027], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 588.7868532252173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 557.574077976966], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.6850274492022], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.9755905674709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 569.83923728527], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 423.09541058313334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 609.6896746165189], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 527.7880286270662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.2803588207797], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.65906034752857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.90853756326095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 449.5837654569756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.6641452108297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.7689328926849], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 498.770441590863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.50514554959426], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.8624654842458], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.9097897715537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.9544570378813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.4111489817221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5212185987317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.5262062926587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.9372505865084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.4913301585447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.3211289617824], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.1247113251569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 524.8573332645819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.6387263730197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 617.9044601153835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 491.84039030061854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.4450859009229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.3560115012395], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.878776081061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.7492124375674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.52622439576373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.69408678252256], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.5798952957598], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.79117929486375], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.7932988360084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.792872338254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 647.6118748516647], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.5680220001173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.63697860116866], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9731646454644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.8237473570492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.8108334656422], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.1638308677572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.6013239892299], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.5707317247616], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 443.1146897398341], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.9220369244747], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6300487111251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.4271990225716], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 500.4818137220986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 527.3092645496879], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.1879776081525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4353057347079], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.87994878804193], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.3949377382753], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.4637803744531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.9473396540226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.8560822813565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 630.7406028012334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.99202148064364], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.0356830700694], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.32594667452827], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.3746657423881], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.67122637563216], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4508123787871], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.6592312184615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.8585099800715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.9736893015839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 637.9122175107984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.1955137797173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 494.7199387952659], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.1828149796918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 601.1289947975267], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.6191771001185], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.3381354583017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 429.20045912894676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.8371536532313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.9270167548373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.9471248494835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.9973054867768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.1462500003859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.72112561180666], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 593.530389915528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.1838541142181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.1934891188854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.4917751343677], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.9697232330274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.2333596548143], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.7673486386564], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.3416397129491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 541.3561451250066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.62311676302494], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.8925710791717], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.1620814376962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.1031905125329], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.8432710498705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 597.9603451276024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.6430527086839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.3471628270505], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.8317489600514], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.839050083348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.5282626287961], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.43238971765805], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.9789349409814], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.2225019092525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.6101465041956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.824918232652], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.4688309145857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.7877852003875], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 482.6542954602548], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 595.0953007820791], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.49141742587625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 558.8870055818687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.0734509395793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.0081276901934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.5329090386708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.3652686202531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.0375817854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 518.6391993179819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 542.0690151842002], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.6301054572286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6283545902703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 533.2235287056707], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 466.030544359565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.8657588521703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.77778397141594], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.4906821300837], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.3969869444828], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.1938130322338], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.995998341848], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.3008050449005], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 535.9263274096863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.6137983573092], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.3386484551024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 528.6181134761199], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.5404899634831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.7783964998134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.72294112061587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.3717443991761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.49037803741504], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.1276304066477], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.0887682739982], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.4324990659869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9704766909796], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.5835883587526], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.1201700574769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.5119849438112], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.505930774511], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.77830300641017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.09830838168074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3751516961829], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.3759350249068], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.8567063908843], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.8473922263536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.0541165301304], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.6980524012438], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0042480933749], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 476.22996376368997], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.388406854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.9420178487572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.72697344472533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.0384597579654], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4331532452838], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.5098129536639], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.9268983185507], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9359781705669], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.4849185399664], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4731835234663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.902686328858], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 508.43807633236105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 515.7254399167086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.999392475016], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.1677136578253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 439.65591364088266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.194782498046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0862397400867], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.60838425139934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.24921735529244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.81250379760434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.87066150264604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.8094003498974], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.55501106888175], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 542.9622092091613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.17469870172033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.40910580320497], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 510.7352947198684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2855419123816], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.8726076692513], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0840283113087], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.9480229133085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Oceania'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 65.5], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 3.68], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.27], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.93], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 1392680589329.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 1.6], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Canberra'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 375908.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Sydney'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'AUS'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 47.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 61], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.0], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 53.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 58000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 16.3], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.74], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 133.775136], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'AUD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 21844756.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 119.8], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 25766605.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 3.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 7741220.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 13.59], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '036'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 12.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 6.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 9.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 113.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', -25.274398], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Australia and New Zealand'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.7], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 48.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 19.6]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: type:airport,icao:YPPH,distCity:10063.648487575283\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runwaysLength,runways,iata. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runways,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('YPPH', ['commercial_flights', 'iata', 'runwaysWidth'], ['runwaysLength', 'runways'], {'difficultyProp': 'hardhard', 'entityDiff:': 'easy', 'context': [['node', ['PASSENGERS'], 'Airport', 'tz_database_timezone', 'Europe/London'], ['node', ['PASSENGERS'], 'Airport', 'commercial_flights', 474576], ['node', ['PASSENGERS'], 'Airport', 'location', [-0.461941, 51.4706]], ['node', ['PASSENGERS'], 'Airport', 'altitude', 83.0], ['node', ['PASSENGERS'], 'Airport', 'dst', 'E'], ['node', ['PASSENGERS'], 'Airport', 'type', 'airport'], ['node', ['PASSENGERS'], 'Airport', 'runwaysLength', 12400.0], ['node', ['PASSENGERS'], 'Airport', 'country', 'United Kingdom'], ['node', ['PASSENGERS'], 'Airport', 'city', 'London'], ['node', ['PASSENGERS'], 'Airport', 'iata', 'LHR'], ['node', ['PASSENGERS'], 'Airport', 'timezone', '0'], ['node', ['PASSENGERS'], 'Airport', 'source', 'OurAirports'], ['node', ['PASSENGERS'], 'Airport', 'runways', 2.0], ['node', ['PASSENGERS'], 'Airport', 'icao', 'EGLL'], ['node', ['PASSENGERS'], 'Airport', 'name', 'London Heathrow Airport'], ['node', ['PASSENGERS'], 'Airport', 'distCity', 25979.20729323409], ['node', ['PASSENGERS'], 'Airport', 'longitude', -0.461941], ['node', ['PASSENGERS'], 'Airport', 'latitude', 51.4706], ['node', ['PASSENGERS'], 'Airport', 'runwaysWidth', 164.0], ['node', ['PASSENGERS'], 'Airport', 'airport_id', 507], ['node', ['OF'], 'City', 'health_care_index', 71.2], ['node', ['OF'], 'City', 'countrycode', 'au'], ['node', ['OF'], 'City', 'accent_name', 'Perth'], ['node', ['OF'], 'City', 'location', [115.861397, -31.952240000000003]], ['node', ['OF'], 'City', 'climate_index', 95.7], ['node', ['OF'], 'City', 'purchasing_power_index', 140.8], ['node', ['OF'], 'City', 'traffic_commute_time_index', 42.4], ['node', ['OF'], 'City', 'cost_of_living_index', 69.6], ['node', ['OF'], 'City', 'country', 'Australia'], ['node', ['OF'], 'City', 'pollution_index', 27.7], ['node', ['OF'], 'City', 'quality_of_life_index', 193.8], ['node', ['OF'], 'City', 'rank', 14], ['node', ['OF'], 'City', 'safety_index', 58.3], ['node', ['OF'], 'City', 'name', 'perth'], ['node', ['OF'], 'City', 'longitude', 115.861397], ['node', ['OF'], 'City', 'property_price_to_income_ratio', 5.5], ['node', ['OF'], 'City', 'latitude', -31.952240000000003], ['node', ['OF'], 'City', 'population', 1446715.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'location', [28.246, -26.1392]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5558.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12825.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'South Africa'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Johannesburg'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JNB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FAOR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'OR Tambo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20930.74996990909], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 28.246], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -26.1392], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 198.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 813], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'location', [57.683601, -20.430201]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 186.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11056.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Mauritius'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Plaisance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MRU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'FIMP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sir Seewoosagur Ramgoolam International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16812384.586103693], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 57.683601], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.430201], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 893], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Pacific/Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'location', [174.792007446, -37.0080986023]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 23.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'Z'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11061.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'New Zealand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Auckland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AKL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '12'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'NZAA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Auckland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15904.7306655827], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 174.792007446], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.0080986023], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2006], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [55.3643989563, 25.252799987800003]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 62.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14432.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United Arab Emirates'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Dubai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '4'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OMDB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dubai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6038.403573841609], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 55.3643989563], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.252799987800003], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2188], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Hong_Kong'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.915001, 22.308901]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 28.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12467.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hong Kong'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VHHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hong Kong International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 24370.906942983245], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.915001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.308901], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3077], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [98.316902, 8.1132]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 82.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Phuket'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HKT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTSP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phuket International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 32003.874082687587], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 98.316902], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 8.1132], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3179], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.0510025024414, 5.9372100830078125]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12402.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kota Kinabalu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WBKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kota Kinabalu International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4818.2885301308415], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.0510025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 5.9372100830078125], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3269], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.65599823, -6.1255698204]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 34.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11221.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jakarta'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CGK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WIII'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Soekarno-Hatta International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19952.873327230303], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.65599823], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -6.1255698204], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3275], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Kuala_Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'location', [101.70999908446998, 2.745579957962]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 69.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13270.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Malaysia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kuala Lumpur'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KUL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WMKK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kuala Lumpur International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 46888.395782021675], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 101.70999908446998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 2.745579957962], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3304], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'location', [103.994003, 1.35019]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 22.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Singapore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SIN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Singapore Changi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16640.33491669021], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 103.994003], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 1.35019], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3316], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [133.90199279785156, -23.806699752807614]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1789.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5858.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Alice Springs'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ASP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Alice Springs Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12301.660108031643], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 133.90199279785156], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.806699752807614], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [153.11700439453125, -27.38419914245605]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9361.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBBN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Brisbane International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13315.566428894339], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 153.11700439453125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -27.38419914245605], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.66666666666666], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3320], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Brisbane'], ['node', ['ROUTE_TO'], 'Airport', 'location', [145.755004883, -16.885799408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6762.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairns'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CNS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBCS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairns International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4315.184422636413], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 145.755004883], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -16.885799408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 104.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3322], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Hobart'], ['node', ['ROUTE_TO'], 'Airport', 'location', [144.843002, -37.673302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 434.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9749.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Melbourne'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YMML'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Melbourne International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18906.748936221637], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 144.843002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -37.673302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3339], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'location', [138.53100600000002, -34.945]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 20.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7795.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Adelaide'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ADL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPAD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Adelaide International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6433.945978837181], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 138.53100600000002], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.945], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3341], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.773002625, -20.712200164799995]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 29.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7480.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Karratha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KTA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Karratha Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8104.177830659545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.773002625], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.712200164799995], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3345], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.461997986, -30.7894001007]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1203.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5249.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kalgoorlie'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KGI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kalgoorlie Boulder Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.461997986], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -30.7894001007], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3346], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [128.707992554, -15.7781000137]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 145.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kununurra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KNX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kununurra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 3872.4122806130536], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 128.707992554], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -15.7781000137], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3347], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.088996887, -22.2355995178]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 19.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9997.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Learmonth'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPLM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Learmonth Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1705.976705317044], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.088996887], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -22.2355995178], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3348], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [118.625999451, -20.3777999878]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5741.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Port Hedland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PHE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPPD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Port Hedland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7483.170510964459], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 118.625999451], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -20.3777999878], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 103.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3349], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Christmas'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.69000244140624, -10.450599670410156]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 916.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6900.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Christmas Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'XCH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPXM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Christmas Island Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.69000244140624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -10.450599670410156], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3353], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [149.19500732421875, -35.30690002441406]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1886.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8140.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Canberra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CBR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSCB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Canberra International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6494.989912504808], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 149.19500732421875], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -35.30690002441406], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 123.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3355], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'location', [151.177001953125, -33.94609832763672]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 21.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9766.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sydney'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SYD'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '10'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YSSY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sydney Kingsford Smith International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9780.015780688975], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 151.177001953125], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.94609832763672], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3361], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.16699981689, -8.7481698989868]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 14.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9790.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denpasar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DPS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ngurah Rai (Bali) International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12218.671924212083], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.16699981689], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.7481698989868], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3940], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'location', [130.87699890136722, -12.41469955444336]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 103.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8002.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Darwin'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DRW'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '9.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPDN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Darwin International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6454.0079836886625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 130.87699890136722], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -12.41469955444336], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 147.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3999], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [122.23200225830078, -17.944700241088867]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 56.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8064.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Broome'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BME'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBRM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Broome International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 1413.2920249796625], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 122.23200225830078], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.944700241088867], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 4319], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.80899810791016, -34.94329833984375]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 233.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4751.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Albany'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ALH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YABA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Albany Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8444.428815978767], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.80899810791016], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -34.94329833984375], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6235], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.822998, -33.684399]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 470.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4393.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Esperance'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'EPR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YESP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Esperance Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20324.669691320036], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.822998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.684399], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 78.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6266], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [114.707001, -28.796101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 121.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4520.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Geraldton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GET'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YGEL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Geraldton Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9214.850207397747], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 114.707001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -28.796101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 88.66666666666667], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6268], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [119.803001404, -23.4178009033]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1724.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6798.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Newman'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ZNE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNWN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Newman Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17766397.24623855], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 119.803001404], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.4178009033], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6310], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [117.745002747, -23.1711006165]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1406.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6995.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Paraburdoo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YPBO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paraburdoo Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8536.853726239466], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 117.745002747], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -23.1711006165], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6314], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [120.208000183, -33.7971992493]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 4719.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ravensthorpe'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'RVT'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YNRV'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Ravensthorpe Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28126.50928224029], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 120.208000183], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.7971992493], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 7577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Makassar'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.276675, -8.757322]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 319.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10826.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Indonesia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Praya'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LOP'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'WADL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lombok International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 5825.787746988739], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.276675], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -8.757322], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 8401], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Australia/Perth'], ['node', ['ROUTE_TO'], 'Airport', 'location', [115.401596069, -33.688423156700004]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 55.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'O'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8071.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Brusselton'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BQB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YBLN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Busselton Regional Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 115.401596069], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -33.688423156700004], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9090], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [123.82800293, -17.5813999176]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 300.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10003.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Australia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Derby'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DCN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'YCIN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'RAAF Base Curtin'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9823191.307539554], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 123.82800293], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -17.5813999176], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 9905], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', '\\N'], ['node', ['ROUTE_TO'], 'Airport', 'location', [51.608056, 25.273056]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 14928.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Qatar'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Doha'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DOH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OTHH'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Hamad International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7672.619050740817], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 51.608056], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.273056], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 11051], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_passengers', 141724], ['edge', ['PASSENGERS'], 'PASSENGERS', 'nb_seats', 170516], ['edge', ['PASSENGERS'], 'PASSENGERS', 'year', 2019], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.2025362631243], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.6812558523761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 508.3855045687054], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.4195670320667], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '346 343'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.9336857336278], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.2602056814085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 587.0011820925939], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.56280175906863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.7159531338469], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.5185846231632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.0121582810918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.83590604039534], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.5730050875931], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.8491382234394], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.5445402122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.5749494563072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.0590134569188], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.9649762790181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.0978963191487], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.1547049279857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.0401004744097], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.164223519384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 622.3973415137738], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 433.527206217356], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 529.331727651091], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.0067937752065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 602.1128873353298], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 503.022373482305], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9100121990385], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.8438264333045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.8284140757384], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.1507764396483], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0272525066715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.3832727304101], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.3853286503956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7918180563407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.7277474100887], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.7825063331557], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.6767583696729], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.87710007569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 599.258508028423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 511.9551573456317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.757888902113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8933704052415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.6972325013244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.4522900590486], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 543.2842554063412], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4762011490865], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.65236227784754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.94371412093443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.7820825772044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.38149947257597], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.5785521775655], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9931946073007], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '763 332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.30833351976105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 422.6451817359142], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H 332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 554.0060133377134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 426.6662319432057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 511.85298147531245], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.1555991084923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4372219567109], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 570.6928092937061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.5567371540335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.90146483345325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.302208945234], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.1743450662014], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.3040499977301], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.92384963415446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 625.1525332976813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.81568949199465], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 522.0320489949096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.4980270116798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.4562370175372], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.08500886824663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2658369157759], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.26118464194604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 509.49828784727066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 427.0114099329785], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2285907391058], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.07926429842854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.9495164918967], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.32065224981454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 526.1890724701276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9643636553158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 491.79350327381326], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8114645776544], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.5502812409808], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.300690686633], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.0654228054552], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.37743438802613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 644.1322969336072], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.401987349898], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 514.0879923327558], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 430.2246147253314], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 560.1843120692553], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.8236016915419], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5279712736085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.8338597725095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3783253674443], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.8907645828957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.0473870510434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.27340288710286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.84193195240096], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.1584360944793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 600.4976321939617], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.52723380292883], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.576665719835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.12928192915075], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.3450509823031], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.62702904718844], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 623.2478869923331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.23676349117994], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.0738477550975], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.17297155461915], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.24125816404575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9878080264196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 553.1233497615187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.7612754438496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.5764996859718], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.3922640167573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.8867020874468], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.769936063906], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.7557861418393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 490.80451159920466], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.812560045748], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.451051921253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 632.2724915064517], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.3971759221196], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.5673321867221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.4832090246027], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 588.7868532252173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 557.574077976966], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.6850274492022], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.9755905674709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 569.83923728527], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 423.09541058313334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 609.6896746165189], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 527.7880286270662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.2803588207797], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.65906034752857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.90853756326095], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 449.5837654569756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.6641452108297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.7689328926849], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 498.770441590863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.50514554959426], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.8624654842458], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 8318948.023348334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.9097897715537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4305'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '343 346'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 497.9544570378813], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5892621.401795073], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 528.4111489817221], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1057'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 525.5212185987317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 553.5262062926587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.9372505865084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5342493.499769855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.4913301585447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.3211289617824], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.1247113251569], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 524.8573332645819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9051663.177504376], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.6387263730197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 617.9044601153835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6042951.963836559], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 491.84039030061854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1680'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CX'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.4450859009229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4837558.770323354], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 530.3560115012395], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.878776081061], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4216516.539859045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.7492124375674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.52622439576373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.69408678252256], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2222'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.5798952957598], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3032441.181729276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.79117929486375], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 646.7932988360084], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.792872338254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2417'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'D7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 647.6118748516647], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 4144796.7420139615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.5680220001173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3378'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MH'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 485.63697860116866], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 556.9731646454644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3021'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '3K'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.8237473570492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.8108334656422], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2183'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'EK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.1638308677572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 456.6013239892299], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2220'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'ET'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.5707317247616], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 443.1146897398341], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.9220369244747], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6300487111251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.4271990225716], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 500.4818137220986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4435'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 527.3092645496879], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 486.1879776081525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4936'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4353057347079], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.87994878804193], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '17891'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.3949377382753], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3915151.444997317], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.4637803744531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '333 772'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.9473396540226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1979739.553109894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.8560822813565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 630.7406028012334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.99202148064364], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.0356830700694], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.32594667452827], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.3746657423881], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.67122637563216], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4508123787871], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3611592.955147056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.6592312184615], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.8585099800715], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3435764.8665458923], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.9736893015839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 637.9122175107984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.1955137797173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 494.7199387952659], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.1828149796918], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 601.1289947975267], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 554.6191771001185], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.3381354583017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 429.20045912894676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.8371536532313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2703768.5111991977], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.9270167548373], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.9471248494835], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.9973054867768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.1462500003859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.72112561180666], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 593.530389915528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2117820.704806353], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 538.1838541142181], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.1934891188854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.4917751343677], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.9697232330274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1252476.902717215], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 558.2333596548143], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.7673486386564], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.3416397129491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 541.3561451250066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.62311676302494], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 571.8925710791717], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 537725.4368380251], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.1620814376962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 547.1031905125329], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.8432710498705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 597.9603451276024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.6430527086839], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.3471628270505], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.8317489600514], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '338'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.839050083348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2213728.712926104], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 559.5282626287961], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.43238971765805], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 432.9789349409814], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 538.2225019092525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 509.6101465041956], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '717 DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 516.824918232652], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1096171.4874376107], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.4688309145857], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 586.7877852003875], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 482.6542954602548], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 595.0953007820791], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1314107.7560978283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.49141742587625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 558.8870055818687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2615451.1625148854], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.0734509395793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E90 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.0081276901934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3088322.703969768], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.5329090386708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 624.3652686202531], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.0375817854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330 763'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 518.6391993179819], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 542.0690151842002], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '321'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.6301054572286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6283545902703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 333 763 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 533.2235287056707], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 466.030544359565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4937'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TT'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 604.8657588521703], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 3280858.4358603344], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.77778397141594], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332 73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.4906821300837], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 6166155.711350348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.3969869444828], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '332'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 592.1938130322338], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.995998341848], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4940'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.3008050449005], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 5331779.633047212], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 535.9263274096863], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4951'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '330'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'TK'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.6137983573092], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.3386484551024], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2520'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'GA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 528.6181134761199], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 552.5404899634831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.7783964998134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 488.72294112061587], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 557.3717443991761], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 438.49037803741504], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2857'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.1276304066477], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2583061.11210757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.0887682739982], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 563.4324990659869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 539.9704766909796], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 583.5835883587526], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2655257.6351714157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.1201700574769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H E90'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.5119849438112], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 494.505930774511], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.77830300641017], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1680430.357050981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.09830838168074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.3751516961829], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.3759350249068], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 591.8567063908843], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 375514.95806932217], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 437.8473922263536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.0541165301304], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.6980524012438], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0042480933749], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 581151.1144426574], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 476.22996376368997], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 619.388406854086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 514.9420178487572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.72697344472533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 513.0384597579654], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4331532452838], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 370331.332978154], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.5098129536639], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.9268983185507], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 450.9359781705669], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.4849185399664], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1021067.3970566533], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.4731835234663], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '73H'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.902686328858], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 508.43807633236105], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 515.7254399167086], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.999392475016], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4089'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.1677136578253], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 991773.631961833], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 439.65591364088266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 539.194782498046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0862397400867], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.60838425139934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 447115.1985802723], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.24921735529244], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.81250379760434], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2580916.984190726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.87066150264604], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3052'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'JQ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 552.8094003498974], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 465.55501106888175], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 542.9622092091613], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 201660.37645790537], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 454.17469870172033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'F50'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.40910580320497], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 510.7352947198684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '345'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'NZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.2855419123816], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1783479.853401746], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.8726076692513], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5360'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320 100'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0840283113087], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 9340088.664547572], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.9480229133085], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4091'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '77L'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'QR'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Oceania'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 65.5], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 3.68], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.27], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.93], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 1392680589329.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 1.6], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Canberra'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 375908.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Sydney'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'AUS'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 47.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 61], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.0], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 53.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 58000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 16.3], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.74], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 133.775136], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'AUD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 21844756.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 119.8], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 25766605.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 3.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 7741220.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 13.59], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'AU'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '036'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 12.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 6.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 9.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 113.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', -25.274398], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Australia and New Zealand'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.7], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 48.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Australia'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 19.6]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:9197.5,runways:2.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: commercial_flights,iata,runwaysWidth. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: commercial_flights,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VEGT', ['distCity', 'altitude', 'city'], ['name', 'latitude', 'source'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'easy', 'context': [['node', ['OF'], 'City', 'region', 3], ['node', ['OF'], 'City', 'countrycode', 'in'], ['node', ['OF'], 'City', 'accent_name', 'Guwahati'], ['node', ['OF'], 'City', 'location', [91.733333, 26.183332999999998]], ['node', ['OF'], 'City', 'name', 'guwahati'], ['node', ['OF'], 'City', 'longitude', 91.733333], ['node', ['OF'], 'City', 'latitude', 26.183332999999998], ['node', ['OF'], 'City', 'country', 'India'], ['node', ['OF'], 'City', 'population', 899132.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [72.8678970337, 19.0886993408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10465.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mumbai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BOM'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VABB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chhatrapati Shivaji International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13408.701795898545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 72.8678970337], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 19.0886993408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2997], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [91.2404022217, 23.8869991302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 46.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7500.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Agartala'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEAT'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Agartala Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6644.301314845647], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 91.2404022217], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8869991302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3038], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.6196975708, 23.8405990601]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1398.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8202.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Aizwal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AJL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VELP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lengpui Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16323.318167219972], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.6196975708], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8405990601], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3039], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.32859802246094, 26.68120002746582]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 412.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9035.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Baghdogra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEBD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Bagdogra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2405.8119868764607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.32859802246094], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.68120002746582], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3040], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.44670104980469, 22.654699325561523]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 16.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9885.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kolkata'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CCU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VECC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Netaji Subhash Chandra Bose International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12331.314894773946], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.44670104980469], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.654699325561523], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3043], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [93.896697998, 24.7600002289]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2540.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9009.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Imphal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IMF'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEIM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Imphal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8295.21283365052], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 93.896697998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.7600002289], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3050], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.9786987305, 24.9129009247]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 352.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5993.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Silchar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Silchar Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20988.48210680302], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.9786987305], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.9129009247], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3055], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [95.0168991089, 27.4839000702]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 362.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mohanbari'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DIB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEMN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dibrugarh Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4108.026577339498], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 95.0168991089], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4839000702], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3057], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.103104, 28.5665]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 777.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12674.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Delhi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VIDP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Indira Gandhi International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15731.797606786417], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.103104], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 28.5665], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3093], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.706299, 13.1979]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3000.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangalore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BLR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VOBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kempegowda International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27354.713400719185], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.706299], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.1979], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3131], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'location', [89.42459869380001, 27.4032001495]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 7332.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7431.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Bhutan'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '6'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VQPR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paro Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19489.592637604343], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 89.42459869380001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4032001495], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3155], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.2017825299622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.9344829733657], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5710574201978], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.42646369196325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.6560164751891], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8465011937379], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.0458517297516], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.4997447368136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0927563940618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4570255825478], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.654781636496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 562.7384805719907], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.8680174680454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.7106516742158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.4837641154695], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 434.9268661195565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.1480133234171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.5589892204403], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.5620168389856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.47839653490644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0306089009449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.55172050874904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.184371649736], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.287527881272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 550.3268854142708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 487.1547393014709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.7879848160503], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 573.6245682645057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 579.1334940671265], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 551.8234820814944], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.2693111132632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.93312010561294], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.699452292528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.78794426817495], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 545.9828737843475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.0607441680479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 633.5981918577681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.00348640078187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.3422378758491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 425.6018731987869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.1327433294625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 473.4588247739408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.7422361258331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.782983949283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.56431586068464], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.6026679149521], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.431422864676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 483.2589112183957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.99234813289894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.72907705274116], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2668737199964], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 441.14381818888063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.1985386874993], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.92382696570303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.1359185463093], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.1106602227431], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.4683475020289], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 471.6289104642136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 606.3752531180065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.1607271540449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 620.5753125105856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 436.9781501981826], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 621.0488250258045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.9342517389279], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.341964585888], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.6792265742308], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4017040473869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.78821768338423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.4550995276144], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.8412497454453], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.4766885810811], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6621470132043], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 641.0920565985074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.4154657959147], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.856099452446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.733538988686], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.9275683640856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.10963547002126], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 500.4532680304257], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5198583645192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 556.9116566670392], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.0989489167378], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 649.0413879733798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.1615858210568], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 568.6311988688932], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9158413032359], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.0158242703391], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.2881394678862], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.30298171274063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9063324525436], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.04257590246806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 472.373943726268], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.86003689346336], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 489.4175112290924], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.3855332749522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.8519597195756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.6833917406806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.8111331913475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5604624929198], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.7756763546319], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 481.04424623119223], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.50571576759313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 488.01246783953457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 517.3113047019681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.9700339786138], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.7141285566078], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.1796595122676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 515.2364518133622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 584.7593530063955], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.7944274010208], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 638.8538655740266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.5085548150297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4582648318899], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.8956375206038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 49.3], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.86], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.36], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.97], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 2611000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.7], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'New Delhi'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 2407672.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Kurebhar'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'IND'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 49.7], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 91], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 11.2], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 34.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 3031000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 23.8], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 2.22], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 78.96288], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'INR'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 471031528.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 180.44], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1366417754.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 29.9], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 3287263.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.3], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 113.0], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '356'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 17.86], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 145.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 28.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 20.593684], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Hindi'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Southern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 69.4], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 60.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 65.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: source:OurAirports,name:Lokpriya Gopinath Bordoloi International Airport,latitude:26.10610008239746\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: distCity,altitude,city. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: distCity,altitude,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VEGT', ['altitude', 'city', 'name'], ['runways', 'runwaysLength', 'iata'], {'difficultyProp': 'easyhard', 'entityDiff:': 'easy', 'context': [['node', ['OF'], 'City', 'region', 3], ['node', ['OF'], 'City', 'countrycode', 'in'], ['node', ['OF'], 'City', 'accent_name', 'Guwahati'], ['node', ['OF'], 'City', 'location', [91.733333, 26.183332999999998]], ['node', ['OF'], 'City', 'name', 'guwahati'], ['node', ['OF'], 'City', 'longitude', 91.733333], ['node', ['OF'], 'City', 'latitude', 26.183332999999998], ['node', ['OF'], 'City', 'country', 'India'], ['node', ['OF'], 'City', 'population', 899132.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [72.8678970337, 19.0886993408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10465.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mumbai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BOM'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VABB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chhatrapati Shivaji International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13408.701795898545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 72.8678970337], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 19.0886993408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2997], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [91.2404022217, 23.8869991302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 46.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7500.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Agartala'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEAT'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Agartala Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6644.301314845647], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 91.2404022217], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8869991302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3038], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.6196975708, 23.8405990601]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1398.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8202.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Aizwal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AJL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VELP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lengpui Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16323.318167219972], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.6196975708], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8405990601], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3039], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.32859802246094, 26.68120002746582]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 412.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9035.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Baghdogra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEBD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Bagdogra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2405.8119868764607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.32859802246094], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.68120002746582], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3040], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.44670104980469, 22.654699325561523]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 16.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9885.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kolkata'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CCU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VECC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Netaji Subhash Chandra Bose International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12331.314894773946], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.44670104980469], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.654699325561523], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3043], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [93.896697998, 24.7600002289]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2540.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9009.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Imphal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IMF'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEIM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Imphal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8295.21283365052], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 93.896697998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.7600002289], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3050], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.9786987305, 24.9129009247]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 352.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5993.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Silchar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Silchar Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20988.48210680302], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.9786987305], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.9129009247], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3055], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [95.0168991089, 27.4839000702]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 362.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mohanbari'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DIB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEMN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dibrugarh Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4108.026577339498], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 95.0168991089], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4839000702], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3057], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.103104, 28.5665]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 777.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12674.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Delhi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VIDP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Indira Gandhi International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15731.797606786417], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.103104], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 28.5665], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3093], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.706299, 13.1979]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3000.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangalore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BLR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VOBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kempegowda International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27354.713400719185], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.706299], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.1979], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3131], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'location', [89.42459869380001, 27.4032001495]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 7332.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7431.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Bhutan'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '6'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VQPR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paro Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19489.592637604343], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 89.42459869380001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4032001495], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3155], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.2017825299622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.9344829733657], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5710574201978], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.42646369196325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.6560164751891], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8465011937379], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.0458517297516], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.4997447368136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0927563940618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4570255825478], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.654781636496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 562.7384805719907], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.8680174680454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.7106516742158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.4837641154695], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 434.9268661195565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.1480133234171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.5589892204403], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.5620168389856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.47839653490644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0306089009449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.55172050874904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.184371649736], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.287527881272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 550.3268854142708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 487.1547393014709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.7879848160503], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 573.6245682645057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 579.1334940671265], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 551.8234820814944], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.2693111132632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.93312010561294], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.699452292528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.78794426817495], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 545.9828737843475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.0607441680479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 633.5981918577681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.00348640078187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.3422378758491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 425.6018731987869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.1327433294625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 473.4588247739408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.7422361258331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.782983949283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.56431586068464], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.6026679149521], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.431422864676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 483.2589112183957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.99234813289894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.72907705274116], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2668737199964], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 441.14381818888063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.1985386874993], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.92382696570303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.1359185463093], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.1106602227431], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.4683475020289], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 471.6289104642136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 606.3752531180065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.1607271540449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 620.5753125105856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 436.9781501981826], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 621.0488250258045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.9342517389279], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.341964585888], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.6792265742308], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4017040473869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.78821768338423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.4550995276144], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.8412497454453], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.4766885810811], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6621470132043], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 641.0920565985074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.4154657959147], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.856099452446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.733538988686], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.9275683640856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.10963547002126], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 500.4532680304257], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5198583645192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 556.9116566670392], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.0989489167378], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 649.0413879733798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.1615858210568], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 568.6311988688932], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9158413032359], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.0158242703391], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.2881394678862], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.30298171274063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9063324525436], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.04257590246806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 472.373943726268], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.86003689346336], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 489.4175112290924], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.3855332749522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.8519597195756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.6833917406806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.8111331913475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5604624929198], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.7756763546319], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 481.04424623119223], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.50571576759313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 488.01246783953457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 517.3113047019681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.9700339786138], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.7141285566078], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.1796595122676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 515.2364518133622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 584.7593530063955], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.7944274010208], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 638.8538655740266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.5085548150297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4582648318899], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.8956375206038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 49.3], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.86], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.36], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.97], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 2611000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.7], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'New Delhi'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 2407672.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Kurebhar'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'IND'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 49.7], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 91], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 11.2], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 34.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 3031000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 23.8], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 2.22], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 78.96288], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'INR'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 471031528.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 180.44], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1366417754.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 29.9], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 3287263.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.3], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 113.0], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '356'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 17.86], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 145.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 28.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 20.593684], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Hindi'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Southern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 69.4], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 60.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 65.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:9000.0,iata:GAU,runways:1.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: altitude,city,name. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,city,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VEGT', ['runwaysLength', 'runwaysWidth', 'commercial_flights'], ['tz_database_timezone', 'country', 'name'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'easy', 'context': [['node', ['OF'], 'City', 'region', 3], ['node', ['OF'], 'City', 'countrycode', 'in'], ['node', ['OF'], 'City', 'accent_name', 'Guwahati'], ['node', ['OF'], 'City', 'location', [91.733333, 26.183332999999998]], ['node', ['OF'], 'City', 'name', 'guwahati'], ['node', ['OF'], 'City', 'longitude', 91.733333], ['node', ['OF'], 'City', 'latitude', 26.183332999999998], ['node', ['OF'], 'City', 'country', 'India'], ['node', ['OF'], 'City', 'population', 899132.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [72.8678970337, 19.0886993408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10465.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mumbai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BOM'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VABB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chhatrapati Shivaji International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13408.701795898545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 72.8678970337], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 19.0886993408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2997], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [91.2404022217, 23.8869991302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 46.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7500.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Agartala'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEAT'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Agartala Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6644.301314845647], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 91.2404022217], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8869991302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3038], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.6196975708, 23.8405990601]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1398.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8202.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Aizwal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AJL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VELP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lengpui Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16323.318167219972], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.6196975708], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8405990601], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3039], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.32859802246094, 26.68120002746582]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 412.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9035.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Baghdogra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEBD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Bagdogra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2405.8119868764607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.32859802246094], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.68120002746582], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3040], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.44670104980469, 22.654699325561523]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 16.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9885.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kolkata'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CCU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VECC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Netaji Subhash Chandra Bose International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12331.314894773946], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.44670104980469], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.654699325561523], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3043], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [93.896697998, 24.7600002289]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2540.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9009.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Imphal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IMF'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEIM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Imphal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8295.21283365052], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 93.896697998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.7600002289], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3050], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.9786987305, 24.9129009247]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 352.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5993.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Silchar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Silchar Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20988.48210680302], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.9786987305], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.9129009247], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3055], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [95.0168991089, 27.4839000702]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 362.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mohanbari'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DIB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEMN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dibrugarh Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4108.026577339498], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 95.0168991089], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4839000702], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3057], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.103104, 28.5665]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 777.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12674.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Delhi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VIDP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Indira Gandhi International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15731.797606786417], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.103104], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 28.5665], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3093], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.706299, 13.1979]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3000.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangalore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BLR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VOBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kempegowda International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27354.713400719185], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.706299], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.1979], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3131], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'location', [89.42459869380001, 27.4032001495]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 7332.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7431.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Bhutan'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '6'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VQPR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paro Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19489.592637604343], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 89.42459869380001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4032001495], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3155], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.2017825299622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.9344829733657], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5710574201978], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.42646369196325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.6560164751891], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8465011937379], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.0458517297516], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.4997447368136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0927563940618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4570255825478], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.654781636496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 562.7384805719907], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.8680174680454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.7106516742158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.4837641154695], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 434.9268661195565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.1480133234171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.5589892204403], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.5620168389856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.47839653490644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0306089009449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.55172050874904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.184371649736], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.287527881272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 550.3268854142708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 487.1547393014709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.7879848160503], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 573.6245682645057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 579.1334940671265], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 551.8234820814944], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.2693111132632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.93312010561294], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.699452292528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.78794426817495], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 545.9828737843475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.0607441680479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 633.5981918577681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.00348640078187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.3422378758491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 425.6018731987869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.1327433294625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 473.4588247739408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.7422361258331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.782983949283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.56431586068464], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.6026679149521], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.431422864676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 483.2589112183957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.99234813289894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.72907705274116], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2668737199964], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 441.14381818888063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.1985386874993], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.92382696570303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.1359185463093], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.1106602227431], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.4683475020289], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 471.6289104642136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 606.3752531180065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.1607271540449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 620.5753125105856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 436.9781501981826], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 621.0488250258045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.9342517389279], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.341964585888], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.6792265742308], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4017040473869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.78821768338423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.4550995276144], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.8412497454453], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.4766885810811], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6621470132043], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 641.0920565985074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.4154657959147], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.856099452446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.733538988686], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.9275683640856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.10963547002126], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 500.4532680304257], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5198583645192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 556.9116566670392], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.0989489167378], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 649.0413879733798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.1615858210568], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 568.6311988688932], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9158413032359], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.0158242703391], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.2881394678862], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.30298171274063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9063324525436], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.04257590246806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 472.373943726268], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.86003689346336], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 489.4175112290924], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.3855332749522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.8519597195756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.6833917406806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.8111331913475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5604624929198], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.7756763546319], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 481.04424623119223], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.50571576759313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 488.01246783953457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 517.3113047019681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.9700339786138], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.7141285566078], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.1796595122676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 515.2364518133622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 584.7593530063955], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.7944274010208], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 638.8538655740266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.5085548150297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4582648318899], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.8956375206038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 49.3], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.86], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.36], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.97], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 2611000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.7], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'New Delhi'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 2407672.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Kurebhar'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'IND'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 49.7], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 91], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 11.2], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 34.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 3031000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 23.8], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 2.22], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 78.96288], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'INR'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 471031528.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 180.44], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1366417754.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 29.9], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 3287263.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.3], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 113.0], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '356'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 17.86], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 145.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 28.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 20.593684], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Hindi'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Southern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 69.4], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 60.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 65.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: tz_database_timezone:Asia/Calcutta,country:India,name:Lokpriya Gopinath Bordoloi International Airport\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runwaysLength,runwaysWidth,commercial_flights. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,runwaysWidth,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VEGT', ['runwaysWidth', 'iata', 'runwaysLength'], ['runways', 'commercial_flights'], {'difficultyProp': 'hardhard', 'entityDiff:': 'easy', 'context': [['node', ['OF'], 'City', 'region', 3], ['node', ['OF'], 'City', 'countrycode', 'in'], ['node', ['OF'], 'City', 'accent_name', 'Guwahati'], ['node', ['OF'], 'City', 'location', [91.733333, 26.183332999999998]], ['node', ['OF'], 'City', 'name', 'guwahati'], ['node', ['OF'], 'City', 'longitude', 91.733333], ['node', ['OF'], 'City', 'latitude', 26.183332999999998], ['node', ['OF'], 'City', 'country', 'India'], ['node', ['OF'], 'City', 'population', 899132.0], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [72.8678970337, 19.0886993408]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10465.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mumbai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BOM'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VABB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chhatrapati Shivaji International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 13408.701795898545], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 72.8678970337], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 19.0886993408], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2997], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [91.2404022217, 23.8869991302]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 46.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7500.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Agartala'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEAT'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Agartala Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 6644.301314845647], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 91.2404022217], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8869991302], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3038], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.6196975708, 23.8405990601]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1398.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8202.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Aizwal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AJL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VELP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Lengpui Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16323.318167219972], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.6196975708], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.8405990601], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3039], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.32859802246094, 26.68120002746582]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 412.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9035.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Baghdogra'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEBD'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Bagdogra Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2405.8119868764607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.32859802246094], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.68120002746582], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3040], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [88.44670104980469, 22.654699325561523]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 16.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9885.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kolkata'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CCU'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VECC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Netaji Subhash Chandra Bose International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 12331.314894773946], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 88.44670104980469], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.654699325561523], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3043], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [93.896697998, 24.7600002289]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2540.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9009.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Imphal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IMF'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEIM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Imphal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8295.21283365052], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 93.896697998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.7600002289], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3050], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [92.9786987305, 24.9129009247]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 352.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 5993.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Silchar'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'IXS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEKU'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Silchar Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20988.48210680302], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 92.9786987305], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 24.9129009247], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3055], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [95.0168991089, 27.4839000702]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 362.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mohanbari'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DIB'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VEMN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Dibrugarh Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4108.026577339498], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 95.0168991089], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4839000702], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3057], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.103104, 28.5665]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 777.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12674.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Delhi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VIDP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Indira Gandhi International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 15731.797606786417], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.103104], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 28.5665], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3093], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Calcutta'], ['node', ['ROUTE_TO'], 'Airport', 'location', [77.706299, 13.1979]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3000.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13123.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'India'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangalore'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BLR'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5.5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VOBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kempegowda International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27354.713400719185], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 77.706299], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.1979], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3131], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'location', [89.42459869380001, 27.4032001495]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 7332.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 7431.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Bhutan'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Thimphu'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PBH'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '6'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VQPR'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Paro Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19489.592637604343], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 89.42459869380001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 27.4032001495], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3155], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'location', [100.74700164794922, 13.681099891662598]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12795.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Thailand'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Bangkok'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'BKK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VTBS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Suvarnabhumi Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 27767.281394397236], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 100.74700164794922], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 13.681099891662598], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3885], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.2017825299622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.9344829733657], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5710574201978], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 446.42646369196325], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.6560164751891], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 531.8465011937379], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 520.0458517297516], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 546.4997447368136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.0927563940618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4570255825478], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 616.654781636496], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 562.7384805719907], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.8680174680454], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.7106516742158], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.4837641154695], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 434.9268661195565], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.1480133234171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.5589892204403], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 559.5620168389856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.47839653490644], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.0306089009449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 504.55172050874904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 627.184371649736], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.287527881272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 550.3268854142708], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 487.1547393014709], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.7879848160503], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 573.6245682645057], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 579.1334940671265], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 551.8234820814944], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 502.2693111132632], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.93312010561294], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.699452292528], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 485.78794426817495], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 545.9828737843475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 496.0607441680479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 633.5981918577681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.00348640078187], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 506.3422378758491], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 425.6018731987869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.1327433294625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 473.4588247739408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.7422361258331], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 484.782983949283], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.56431586068464], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 458.6026679149521], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 594.431422864676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 483.2589112183957], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.99234813289894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.72907705274116], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 582.2668737199964], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2073389.203133334], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 441.14381818888063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 612.1985386874993], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.92382696570303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 551.1359185463093], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 249476.07694449098], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.1106602227431], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.4683475020289], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 471.6289104642136], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 606.3752531180065], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 272915.4243603943], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.1607271540449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 620.5753125105856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 436.9781501981826], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 621.0488250258045], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 331042.4399810757], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.9342517389279], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.341964585888], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.6792265742308], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 570.4017040473869], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 452.78821768338423], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 605.4550995276144], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 574.8412497454453], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319 320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 639.4766885810811], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 534.6621470132043], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 641.0920565985074], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 519.4154657959147], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.856099452446], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 498870.4370499662], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.733538988686], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 546.9275683640856], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 467.10963547002126], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 500.4532680304257], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5198583645192], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 556.9116566670392], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.0989489167378], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 649.0413879733798], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 276433.5435239303], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 506.1615858210568], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 568.6311988688932], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9158413032359], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 534.0158242703391], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 481.2881394678862], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'ATR'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.30298171274063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 192930.44730884064], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.9063324525436], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.04257590246806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 472.373943726268], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 484.86003689346336], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 489.4175112290924], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 555.3855332749522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 373815.9946258442], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 474.8519597195756], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 521.6833917406806], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 463.8111331913475], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5604624929198], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.7756763546319], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3000'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '9W'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 481.04424623119223], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.50571576759313], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '218'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AI'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 488.01246783953457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 517.3113047019681], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2575'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G8'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.9700339786138], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.7141285566078], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '241'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'S2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 567.1796595122676], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1457141.7604957246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 515.2364518133622], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '4375'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'SG'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 584.7593530063955], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 2041672.737926408], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.7944274010208], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2850'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', '6E'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 638.8538655740266], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 258843.1546728397], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.5085548150297], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.4582648318899], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1681494.945345348], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 567.8956375206038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2058'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KB'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 49.3], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.86], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 5.36], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.97], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 2611000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.7], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'New Delhi'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 2407672.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Kurebhar'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'IND'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 49.7], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 91], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 11.2], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 34.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 3031000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 23.8], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 2.22], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 78.96288], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'INR'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 471031528.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 180.44], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1366417754.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 29.9], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 3287263.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.3], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'IN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 113.0], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '356'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 17.86], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 145.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 28.1], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 20.593684], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Hindi'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Southern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 69.4], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 60.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'India'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 65.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runways:1.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runwaysWidth,iata,runwaysLength. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysWidth,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ZUBJ', ['icao', 'longitude', 'name'], ['dst', 'altitude'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.81099700927734, 22.639299392700195]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shenzhen'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SZX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGSZ'], ['node', ['ROUTE_TO'], 'Airport', 'name', "Shenzhen Bao'an International Airport"], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 35167.56373759736], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.81099700927734], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.639299392700195], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3374], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [102.9291667, 25.1019444]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6903.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13943.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kunming'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KMG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZPPP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kunming Changshui International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 22387.61104349984], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 102.9291667], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.1019444], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3382], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.33599853515624, 31.19790077209473]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10990.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SHA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shanghai Hongqiao International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18013.423854958113], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.33599853515624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 31.19790077209473], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3391], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.64199829101562, 29.719200134277344]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1365.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Chongqing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUCK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chongqing Jiangbei International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19435.10118602196], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.64199829101562], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 29.719200134277344], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3393], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.8010025024414, 26.53849983215332]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3736.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guiyang'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KWE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUGY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Longdongbao Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9768.50677608875], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.8010025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.53849983215332], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3394], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.38800048828124, 39.78279876708984]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 0.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Beijing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'NAY'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZBNY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Beijing Nanyuan Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 712915.6066670679], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.38800048828124], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.78279876708984], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6341], ['node', ['OF'], 'City', 'region', 18], ['node', ['OF'], 'City', 'countrycode', 'cn'], ['node', ['OF'], 'City', 'accent_name', 'Bijie'], ['node', ['OF'], 'City', 'location', [105.29416699999999, 27.3125]], ['node', ['OF'], 'City', 'name', 'bijie'], ['node', ['OF'], 'City', 'longitude', 105.29416699999999], ['node', ['OF'], 'City', 'latitude', 27.3125], ['node', ['OF'], 'City', 'country', 'China'], ['node', ['OF'], 'City', 'population', 77645.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.697516283684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.7037797593821], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.822141048573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.04467624055536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.2150321717166], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.78829101717173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.352806827402], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.3913963911868], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.9936417181606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63602640903525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.76481885120404], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.1392618207726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.9116423592236], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.06548509113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.8913499461447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63161114028134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.4660130234596], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.4769788840799], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.4838345240642], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.169830004673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 504.5374940957556], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.7894252017123], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5199280379122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.5590974860769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.6356162184184], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.1850296612674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.7796536526904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.10863339045284], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.9390151087705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 428.8820692733705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.8483287594335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 544.4326626101001], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 68.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 1.98], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 4.32], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.96], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 19910000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.9], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Beijing'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 9893038.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Shanghai'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'CHN'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 59.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 86], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.4], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 30.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 2695000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 22.4], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.69], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 104.195397], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'CNY'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 842933962.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 125.08], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1397715000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 7.4], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9596960.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.87], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.2], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '156'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.9], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 29.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 50.6], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 35.86166], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Standard Chinese'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Eastern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 77.0], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 56.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 32.4]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: altitude:4751.0,dst:N\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: icao,longitude,name. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,longitude,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ZUBJ', ['latitude', 'location', 'timezone'], ['iata', 'runways', 'runwaysLength'], {'difficultyProp': 'easyhard', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.81099700927734, 22.639299392700195]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shenzhen'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SZX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGSZ'], ['node', ['ROUTE_TO'], 'Airport', 'name', "Shenzhen Bao'an International Airport"], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 35167.56373759736], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.81099700927734], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.639299392700195], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3374], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [102.9291667, 25.1019444]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6903.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13943.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kunming'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KMG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZPPP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kunming Changshui International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 22387.61104349984], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 102.9291667], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.1019444], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3382], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.33599853515624, 31.19790077209473]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10990.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SHA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shanghai Hongqiao International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18013.423854958113], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.33599853515624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 31.19790077209473], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3391], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.64199829101562, 29.719200134277344]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1365.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Chongqing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUCK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chongqing Jiangbei International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19435.10118602196], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.64199829101562], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 29.719200134277344], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3393], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.8010025024414, 26.53849983215332]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3736.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guiyang'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KWE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUGY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Longdongbao Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9768.50677608875], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.8010025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.53849983215332], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3394], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.38800048828124, 39.78279876708984]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 0.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Beijing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'NAY'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZBNY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Beijing Nanyuan Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 712915.6066670679], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.38800048828124], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.78279876708984], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6341], ['node', ['OF'], 'City', 'region', 18], ['node', ['OF'], 'City', 'countrycode', 'cn'], ['node', ['OF'], 'City', 'accent_name', 'Bijie'], ['node', ['OF'], 'City', 'location', [105.29416699999999, 27.3125]], ['node', ['OF'], 'City', 'name', 'bijie'], ['node', ['OF'], 'City', 'longitude', 105.29416699999999], ['node', ['OF'], 'City', 'latitude', 27.3125], ['node', ['OF'], 'City', 'country', 'China'], ['node', ['OF'], 'City', 'population', 77645.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.697516283684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.7037797593821], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.822141048573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.04467624055536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.2150321717166], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.78829101717173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.352806827402], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.3913963911868], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.9936417181606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63602640903525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.76481885120404], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.1392618207726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.9116423592236], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.06548509113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.8913499461447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63161114028134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.4660130234596], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.4769788840799], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.4838345240642], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.169830004673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 504.5374940957556], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.7894252017123], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5199280379122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.5590974860769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.6356162184184], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.1850296612674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.7796536526904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.10863339045284], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.9390151087705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 428.8820692733705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.8483287594335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 544.4326626101001], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 68.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 1.98], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 4.32], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.96], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 19910000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.9], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Beijing'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 9893038.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Shanghai'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'CHN'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 59.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 86], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.4], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 30.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 2695000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 22.4], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.69], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 104.195397], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'CNY'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 842933962.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 125.08], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1397715000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 7.4], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9596960.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.87], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.2], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '156'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.9], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 29.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 50.6], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 35.86166], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Standard Chinese'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Eastern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 77.0], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 56.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 32.4]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:8530.0,iata:BFJ,runways:1.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: latitude,location,timezone. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: latitude,location,timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ZUBJ', ['runways', 'runwaysWidth', 'iata'], ['city', 'icao', 'distCity'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.81099700927734, 22.639299392700195]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shenzhen'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SZX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGSZ'], ['node', ['ROUTE_TO'], 'Airport', 'name', "Shenzhen Bao'an International Airport"], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 35167.56373759736], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.81099700927734], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.639299392700195], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3374], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [102.9291667, 25.1019444]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6903.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13943.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kunming'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KMG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZPPP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kunming Changshui International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 22387.61104349984], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 102.9291667], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.1019444], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3382], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.33599853515624, 31.19790077209473]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10990.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SHA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shanghai Hongqiao International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18013.423854958113], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.33599853515624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 31.19790077209473], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3391], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.64199829101562, 29.719200134277344]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1365.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Chongqing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUCK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chongqing Jiangbei International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19435.10118602196], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.64199829101562], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 29.719200134277344], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3393], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.8010025024414, 26.53849983215332]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3736.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guiyang'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KWE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUGY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Longdongbao Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9768.50677608875], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.8010025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.53849983215332], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3394], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.38800048828124, 39.78279876708984]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 0.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Beijing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'NAY'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZBNY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Beijing Nanyuan Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 712915.6066670679], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.38800048828124], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.78279876708984], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6341], ['node', ['OF'], 'City', 'region', 18], ['node', ['OF'], 'City', 'countrycode', 'cn'], ['node', ['OF'], 'City', 'accent_name', 'Bijie'], ['node', ['OF'], 'City', 'location', [105.29416699999999, 27.3125]], ['node', ['OF'], 'City', 'name', 'bijie'], ['node', ['OF'], 'City', 'longitude', 105.29416699999999], ['node', ['OF'], 'City', 'latitude', 27.3125], ['node', ['OF'], 'City', 'country', 'China'], ['node', ['OF'], 'City', 'population', 77645.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.697516283684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.7037797593821], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.822141048573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.04467624055536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.2150321717166], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.78829101717173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.352806827402], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.3913963911868], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.9936417181606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63602640903525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.76481885120404], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.1392618207726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.9116423592236], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.06548509113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.8913499461447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63161114028134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.4660130234596], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.4769788840799], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.4838345240642], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.169830004673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 504.5374940957556], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.7894252017123], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5199280379122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.5590974860769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.6356162184184], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.1850296612674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.7796536526904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.10863339045284], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.9390151087705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 428.8820692733705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.8483287594335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 544.4326626101001], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 68.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 1.98], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 4.32], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.96], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 19910000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.9], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Beijing'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 9893038.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Shanghai'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'CHN'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 59.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 86], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.4], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 30.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 2695000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 22.4], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.69], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 104.195397], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'CNY'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 842933962.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 125.08], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1397715000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 7.4], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9596960.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.87], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.2], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '156'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.9], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 29.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 50.6], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 35.86166], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Standard Chinese'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Eastern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 77.0], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 56.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 32.4]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: city:Bijie,icao:ZUBJ,distCity:18314.734739348147\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,runwaysWidth,iata. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysWidth,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ZUBJ', ['iata', 'runwaysLength', 'commercial_flights'], ['runways', 'runwaysWidth'], {'difficultyProp': 'hardhard', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.29900360107422, 23.39240074157715]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 50.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11975.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guangzhou'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGGG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Guangzhou Baiyun International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 31101.038787102378], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.29900360107422], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 23.39240074157715], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3370], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [113.81099700927734, 22.639299392700195]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shenzhen'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SZX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZGSZ'], ['node', ['ROUTE_TO'], 'Airport', 'name', "Shenzhen Bao'an International Airport"], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 35167.56373759736], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 113.81099700927734], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.639299392700195], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3374], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [102.9291667, 25.1019444]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6903.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 13943.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Kunming'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KMG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZPPP'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Kunming Changshui International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 22387.61104349984], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 102.9291667], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 25.1019444], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3382], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [121.33599853515624, 31.19790077209473]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 10.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10990.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SHA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZSSS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Shanghai Hongqiao International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 18013.423854958113], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 121.33599853515624], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 31.19790077209473], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 172.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3391], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.64199829101562, 29.719200134277344]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1365.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Chongqing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CKG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUCK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Chongqing Jiangbei International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19435.10118602196], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.64199829101562], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 29.719200134277344], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 160.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3393], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.8010025024414, 26.53849983215332]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 3736.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11811.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Guiyang'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'KWE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZUGY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Longdongbao Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 9768.50677608875], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.8010025024414], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 26.53849983215332], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3394], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Shanghai'], ['node', ['ROUTE_TO'], 'Airport', 'location', [116.38800048828124, 39.78279876708984]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 0.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'country', 'China'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Beijing'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'NAY'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ZBNY'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Beijing Nanyuan Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 712915.6066670679], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 116.38800048828124], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.78279876708984], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6341], ['node', ['OF'], 'City', 'region', 18], ['node', ['OF'], 'City', 'countrycode', 'cn'], ['node', ['OF'], 'City', 'accent_name', 'Bijie'], ['node', ['OF'], 'City', 'location', [105.29416699999999, 27.3125]], ['node', ['OF'], 'City', 'name', 'bijie'], ['node', ['OF'], 'City', 'longitude', 105.29416699999999], ['node', ['OF'], 'City', 'latitude', 27.3125], ['node', ['OF'], 'City', 'country', 'China'], ['node', ['OF'], 'City', 'population', 77645.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.697516283684], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 462.7037797593821], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.822141048573], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.04467624055536], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 635.2150321717166], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.78829101717173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.352806827402], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.3913963911868], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 577.9936417181606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63602640903525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 503.76481885120404], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.1392618207726], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 530.9116423592236], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 522.06548509113], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.8913499461447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 455.63161114028134], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 510.4660130234596], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 897586.6369975413], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 507.4769788840799], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.4838345240642], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 986343.3274686515], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.169830004673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1767'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'CZ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 504.5374940957556], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 350150.8140326764], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 464.7894252017123], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.5199280379122], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1600278.2485981672], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.5590974860769], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3081'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'HO'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 493.6356162184184], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 295989.36668493855], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 540.1850296612674], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 614.7796536526904], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 154857.83761271066], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.10863339045284], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15837'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR2 CR9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G5'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.9390151087705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 428.8820692733705], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '1758'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MU'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 628.8483287594335], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1719559.771684942], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 544.4326626101001], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '10741'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '737'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'KN'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 68.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 1.98], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 4.32], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.96], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 19910000000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.9], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Beijing'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 9893038.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Shanghai'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'CHN'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 59.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 86], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.4], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 30.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 2695000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 22.4], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.69], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 104.195397], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'CNY'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 842933962.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 125.08], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 1397715000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 7.4], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9596960.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 0.87], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'CN'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.2], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '156'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.9], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 29.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 142.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 50.6], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 35.86166], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Standard Chinese'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Eastern Asia'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 77.0], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 56.2], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'China'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 32.4]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runways:1.0,runwaysWidth:148.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: iata,runwaysLength,commercial_flights. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('KEUG', ['altitude', 'icao', 'tz_database_timezone'], ['distCity', 'name', 'dst'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.221001, 37.721298]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 9.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6261.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oakland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OAK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KOAK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Metropolitan Oakland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4091485.0264444626], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.221001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.721298], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 131.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3453], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.375, 37.61899948120117]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9655.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'San Francisco'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SFO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSFO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'San Francisco International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17780.42048042369], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.375], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.61899948120117], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 200.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3469], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-118.4079971, 33.94250107]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 125.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10599.25], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Los Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Los Angeles International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19536.969580216755], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -118.4079971], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.94250107], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 162.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3484], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.97799682617188, 40.78839874267578]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 4227.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9623.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Salt Lake City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SLC'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSLC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Salt Lake City International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8006.163303059674], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.97799682617188], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 40.78839874267578], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3536], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.308998, 47.449001]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 433.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10250.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Seattle'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSEA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Seattle Tacoma International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17597.05802431977], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.308998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 47.449001], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.5979996, 45.58869934]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 31.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8667.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Portland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PDX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KPDX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Portland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2972064.4473019396], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.5979996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 45.58869934], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3720], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-104.672996521, 39.861698150635]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5431.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12666.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denver'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KDEN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Denver International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 29904.603669261607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -104.672996521], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.861698150635], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 158.33333333333334], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3751], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-115.1520004, 36.08010101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2181.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10950.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Las Vegas'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'McCarran International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 897256.5081144588], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -115.1520004], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 36.08010101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3877], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Phoenix'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.6549988, 33.30780029]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9967.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mesa'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AZA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KIWA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phoenix-Mesa-Gateway Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20078.649943487668], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.6549988], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.30780029], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6505], ['node', ['OF'], 'City', 'countrycode', 'us'], ['node', ['OF'], 'City', 'accent_name', 'Eugene'], ['node', ['OF'], 'City', 'location', [-123.08555559999999, 44.0522222]], ['node', ['OF'], 'City', 'name', 'eugene'], ['node', ['OF'], 'City', 'longitude', -123.08555559999999], ['node', ['OF'], 'City', 'latitude', 44.0522222], ['node', ['OF'], 'City', 'country', 'United States of America'], ['node', ['OF'], 'City', 'population', 145208.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.0336883135337], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 561.9703167293428], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.2432114397732], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0618286674929], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 548.9924326199261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.56733046502274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 564.9928107813444], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.1025482708467], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.1969115013167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.51531733771986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.9551965024157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 548.3391244247754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR7 CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 517.3158401694254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.2750758465131], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 650.6829480251962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 532.1425597514063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 585.2411277479894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.49470153441627], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.1938934094276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.64825387770367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.1038813430056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.5272657870831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.5738156277788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 549.892960872306], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 636.0949856219415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.7803853999984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 565.2095993807518], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.6449504758046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.1195401657099], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.169482189261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.6872796351291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.6586560634053], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.7606100643367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.5924192724859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.1185499915416], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.2319197516696], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.0999455161541], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5486253900035], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.7623671111876], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.14086169456033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.8855683936606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.2074367879787], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.0298608594414], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.7514739796788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.2285143779793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.6405152974688], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4342937996171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.6827577496625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.3785761510286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.4441396949752], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.998559174479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6786549535618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.8371816535318], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.12443003050197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Americas'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 62.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.61], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 14.7], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.71], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 21427700000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.5], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Washington, D.C.'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 5006302.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:US'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'New York City'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'USA'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.6], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 1], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.6], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 21.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 1359000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'United States of America'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.9], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.73], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', -95.712891], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'USD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 270663028.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 117.24], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 328239523.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 5.6], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9833517.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 7.25], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'US'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 101.8], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '840'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 11.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 88.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 37.09024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern America'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 78.5], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'United States'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 11.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: dst:A,name:Mahlon Sweet Field,distCity:12927.753168152423\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: altitude,icao,tz_database_timezone. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: altitude,icao,tz_database_timezone. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('KEUG', ['timezone', 'name', 'altitude'], ['iata', 'runwaysLength', 'runways'], {'difficultyProp': 'easyhard', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.221001, 37.721298]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 9.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6261.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oakland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OAK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KOAK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Metropolitan Oakland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4091485.0264444626], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.221001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.721298], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 131.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3453], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.375, 37.61899948120117]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9655.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'San Francisco'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SFO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSFO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'San Francisco International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17780.42048042369], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.375], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.61899948120117], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 200.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3469], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-118.4079971, 33.94250107]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 125.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10599.25], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Los Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Los Angeles International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19536.969580216755], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -118.4079971], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.94250107], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 162.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3484], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.97799682617188, 40.78839874267578]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 4227.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9623.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Salt Lake City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SLC'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSLC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Salt Lake City International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8006.163303059674], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.97799682617188], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 40.78839874267578], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3536], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.308998, 47.449001]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 433.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10250.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Seattle'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSEA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Seattle Tacoma International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17597.05802431977], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.308998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 47.449001], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.5979996, 45.58869934]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 31.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8667.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Portland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PDX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KPDX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Portland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2972064.4473019396], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.5979996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 45.58869934], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3720], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-104.672996521, 39.861698150635]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5431.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12666.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denver'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KDEN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Denver International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 29904.603669261607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -104.672996521], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.861698150635], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 158.33333333333334], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3751], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-115.1520004, 36.08010101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2181.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10950.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Las Vegas'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'McCarran International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 897256.5081144588], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -115.1520004], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 36.08010101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3877], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Phoenix'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.6549988, 33.30780029]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9967.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mesa'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AZA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KIWA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phoenix-Mesa-Gateway Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20078.649943487668], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.6549988], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.30780029], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6505], ['node', ['OF'], 'City', 'countrycode', 'us'], ['node', ['OF'], 'City', 'accent_name', 'Eugene'], ['node', ['OF'], 'City', 'location', [-123.08555559999999, 44.0522222]], ['node', ['OF'], 'City', 'name', 'eugene'], ['node', ['OF'], 'City', 'longitude', -123.08555559999999], ['node', ['OF'], 'City', 'latitude', 44.0522222], ['node', ['OF'], 'City', 'country', 'United States of America'], ['node', ['OF'], 'City', 'population', 145208.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.0336883135337], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 561.9703167293428], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.2432114397732], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0618286674929], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 548.9924326199261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.56733046502274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 564.9928107813444], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.1025482708467], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.1969115013167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.51531733771986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.9551965024157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 548.3391244247754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR7 CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 517.3158401694254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.2750758465131], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 650.6829480251962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 532.1425597514063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 585.2411277479894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.49470153441627], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.1938934094276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.64825387770367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.1038813430056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.5272657870831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.5738156277788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 549.892960872306], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 636.0949856219415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.7803853999984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 565.2095993807518], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.6449504758046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.1195401657099], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.169482189261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.6872796351291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.6586560634053], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.7606100643367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.5924192724859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.1185499915416], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.2319197516696], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.0999455161541], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5486253900035], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.7623671111876], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.14086169456033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.8855683936606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.2074367879787], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.0298608594414], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.7514739796788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.2285143779793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.6405152974688], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4342937996171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.6827577496625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.3785761510286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.4441396949752], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.998559174479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6786549535618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.8371816535318], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.12443003050197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Americas'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 62.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.61], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 14.7], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.71], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 21427700000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.5], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Washington, D.C.'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 5006302.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:US'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'New York City'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'USA'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.6], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 1], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.6], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 21.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 1359000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'United States of America'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.9], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.73], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', -95.712891], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'USD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 270663028.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 117.24], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 328239523.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 5.6], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9833517.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 7.25], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'US'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 101.8], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '840'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 11.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 88.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 37.09024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern America'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 78.5], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'United States'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 11.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:6412.333333333333,iata:EUG,runways:3.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: timezone,name,altitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,name,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('KEUG', ['runways', 'iata', 'runwaysLength'], ['dst', 'name', 'icao'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.221001, 37.721298]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 9.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6261.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oakland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OAK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KOAK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Metropolitan Oakland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4091485.0264444626], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.221001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.721298], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 131.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3453], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.375, 37.61899948120117]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9655.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'San Francisco'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SFO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSFO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'San Francisco International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17780.42048042369], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.375], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.61899948120117], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 200.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3469], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-118.4079971, 33.94250107]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 125.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10599.25], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Los Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Los Angeles International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19536.969580216755], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -118.4079971], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.94250107], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 162.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3484], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.97799682617188, 40.78839874267578]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 4227.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9623.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Salt Lake City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SLC'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSLC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Salt Lake City International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8006.163303059674], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.97799682617188], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 40.78839874267578], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3536], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.308998, 47.449001]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 433.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10250.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Seattle'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSEA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Seattle Tacoma International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17597.05802431977], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.308998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 47.449001], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.5979996, 45.58869934]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 31.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8667.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Portland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PDX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KPDX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Portland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2972064.4473019396], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.5979996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 45.58869934], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3720], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-104.672996521, 39.861698150635]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5431.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12666.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denver'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KDEN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Denver International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 29904.603669261607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -104.672996521], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.861698150635], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 158.33333333333334], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3751], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-115.1520004, 36.08010101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2181.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10950.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Las Vegas'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'McCarran International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 897256.5081144588], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -115.1520004], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 36.08010101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3877], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Phoenix'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.6549988, 33.30780029]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9967.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mesa'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AZA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KIWA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phoenix-Mesa-Gateway Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20078.649943487668], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.6549988], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.30780029], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6505], ['node', ['OF'], 'City', 'countrycode', 'us'], ['node', ['OF'], 'City', 'accent_name', 'Eugene'], ['node', ['OF'], 'City', 'location', [-123.08555559999999, 44.0522222]], ['node', ['OF'], 'City', 'name', 'eugene'], ['node', ['OF'], 'City', 'longitude', -123.08555559999999], ['node', ['OF'], 'City', 'latitude', 44.0522222], ['node', ['OF'], 'City', 'country', 'United States of America'], ['node', ['OF'], 'City', 'population', 145208.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.0336883135337], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 561.9703167293428], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.2432114397732], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0618286674929], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 548.9924326199261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.56733046502274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 564.9928107813444], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.1025482708467], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.1969115013167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.51531733771986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.9551965024157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 548.3391244247754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR7 CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 517.3158401694254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.2750758465131], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 650.6829480251962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 532.1425597514063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 585.2411277479894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.49470153441627], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.1938934094276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.64825387770367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.1038813430056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.5272657870831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.5738156277788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 549.892960872306], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 636.0949856219415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.7803853999984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 565.2095993807518], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.6449504758046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.1195401657099], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.169482189261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.6872796351291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.6586560634053], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.7606100643367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.5924192724859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.1185499915416], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.2319197516696], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.0999455161541], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5486253900035], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.7623671111876], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.14086169456033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.8855683936606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.2074367879787], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.0298608594414], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.7514739796788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.2285143779793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.6405152974688], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4342937996171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.6827577496625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.3785761510286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.4441396949752], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.998559174479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6786549535618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.8371816535318], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.12443003050197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Americas'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 62.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.61], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 14.7], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.71], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 21427700000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.5], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Washington, D.C.'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 5006302.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:US'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'New York City'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'USA'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.6], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 1], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.6], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 21.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 1359000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'United States of America'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.9], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.73], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', -95.712891], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'USD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 270663028.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 117.24], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 328239523.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 5.6], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9833517.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 7.25], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'US'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 101.8], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '840'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 11.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 88.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 37.09024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern America'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 78.5], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'United States'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 11.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: dst:A,icao:KEUG,name:Mahlon Sweet Field\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,iata,runwaysLength. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('KEUG', ['runways', 'commercial_flights', 'runwaysLength'], ['runwaysWidth', 'iata'], {'difficultyProp': 'hardhard', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.221001, 37.721298]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 9.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 6261.75], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oakland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OAK'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KOAK'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Metropolitan Oakland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4091485.0264444626], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.221001], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.721298], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 131.25], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3453], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.375, 37.61899948120117]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 13.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9655.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'San Francisco'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SFO'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSFO'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'San Francisco International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17780.42048042369], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.375], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 37.61899948120117], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 200.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3469], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-118.4079971, 33.94250107]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 125.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10599.25], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Los Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Los Angeles International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19536.969580216755], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -118.4079971], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.94250107], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 162.5], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3484], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.97799682617188, 40.78839874267578]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 4227.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9623.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Salt Lake City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SLC'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSLC'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Salt Lake City International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 8006.163303059674], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.97799682617188], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 40.78839874267578], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3536], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.308998, 47.449001]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 433.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10250.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Seattle'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SEA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KSEA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Seattle Tacoma International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 17597.05802431977], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.308998], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 47.449001], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3577], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-122.5979996, 45.58869934]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 31.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 8667.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Portland'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'PDX'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KPDX'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Portland International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2972064.4473019396], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -122.5979996], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 45.58869934], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3720], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Denver'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-104.672996521, 39.861698150635]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 5431.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12666.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Denver'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'DEN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KDEN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Denver International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 29904.603669261607], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -104.672996521], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 39.861698150635], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 158.33333333333334], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3751], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Los_Angeles'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-115.1520004, 36.08010101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 2181.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'A'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10950.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Las Vegas'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'LAS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-8'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 4.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KLAS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'McCarran International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 897256.5081144588], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -115.1520004], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 36.08010101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3877], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'America/Phoenix'], ['node', ['ROUTE_TO'], 'Airport', 'location', [-111.6549988, 33.30780029]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'N'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9967.666666666666], ['node', ['ROUTE_TO'], 'Airport', 'country', 'United States'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Mesa'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'AZA'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '-7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'KIWA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Phoenix-Mesa-Gateway Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 20078.649943487668], ['node', ['ROUTE_TO'], 'Airport', 'longitude', -111.6549988], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 33.30780029], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 150.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6505], ['node', ['OF'], 'City', 'countrycode', 'us'], ['node', ['OF'], 'City', 'accent_name', 'Eugene'], ['node', ['OF'], 'City', 'location', [-123.08555559999999, 44.0522222]], ['node', ['OF'], 'City', 'name', 'eugene'], ['node', ['OF'], 'City', 'longitude', -123.08555559999999], ['node', ['OF'], 'City', 'latitude', 44.0522222], ['node', ['OF'], 'City', 'country', 'United States of America'], ['node', ['OF'], 'City', 'population', 145208.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 487.0336883135337], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 561.9703167293428], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.2432114397732], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 520.0618286674929], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 548.9924326199261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 440.56733046502274], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 564.9928107813444], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 525.1025482708467], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 626.1969115013167], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 442.51531733771986], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.9551965024157], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 548.3391244247754], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CR7 CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 517.3158401694254], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 572.2750758465131], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 650.6829480251962], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 532.1425597514063], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 585.2411277479894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 492.49470153441627], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 513.1938934094276], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 453.64825387770367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 562.1038813430056], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 523.5272657870831], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.5738156277788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 549.892960872306], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 636.0949856219415], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 444.7803853999984], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 565.2095993807518], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 451.6449504758046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 512.1195401657099], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 717653.2487698044], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 533.169482189261], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 489.6872796351291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 727607.6812922246], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 498.6586560634053], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.7606100643367], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 568.5924192724859], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '24'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 590.1185499915416], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 445.2319197516696], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 608.0999455161541], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1206612.9169556492], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 468.5486253900035], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5265'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'US'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 549.7623671111876], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 993542.5341281894], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.14086169456033], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2009'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'DL'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.8855683936606], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 376643.32015451125], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 537.2074367879787], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 561.0298608594414], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.7514739796788], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'AS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 507.2285143779793], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 170030.78573934291], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 550.6405152974688], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'EM2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 648.4342937996171], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 495.6827577496625], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5209'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'CRJ CR7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'UA'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 523.3785761510286], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1601525.5511949], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 501.4441396949752], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2468'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '319'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'F9'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 580.998559174479], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1127025.9065358127], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 565.6786549535618], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 603.8371816535318], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 1564424.5224277067], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 457.12443003050197], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '35'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'M80'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'G4'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Americas'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 62.0], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.61], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 14.7], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.71], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 21427700000000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 7.5], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Washington, D.C.'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 5006302.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:US'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'New York City'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'USA'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.6], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 1], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 9.6], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 21.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 1359000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'United States of America'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.9], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.73], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', -95.712891], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'USD'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 270663028.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 117.24], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 328239523.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 5.6], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 9833517.0], ['node', ['OF', 'City', 'IN'], 'Country', 'minimum_wage', 7.25], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'US'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 101.8], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '840'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 11.6], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 19.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 88.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 37.09024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'None'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern America'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 78.5], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'United States'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 11.1]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: iata:EUG,runwaysWidth:150.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,commercial_flights,runwaysLength. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,runwaysLength. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('HESN', ['tz_database_timezone', 'distCity', 'latitude'], ['timezone', 'icao'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.611700058, 22.3759994507]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 616.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Abu Simbel'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ABS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HEBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Abu Simbel Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2457.370449965094], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.611700058], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.3759994507], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1127], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.40559959411621, 30.12190055847168]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12358.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HECA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16990.960849098446], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.40559959411621], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 30.12190055847168], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1128], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Riyadh'], ['node', ['ROUTE_TO'], 'Airport', 'location', [39.156502, 21.6796]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 48.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12904.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Saudi Arabia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jeddah'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JED'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OEJN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'King Abdulaziz International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19233.42631403247], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 39.156502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.6796], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2072], ['node', ['OF'], 'City', 'region', 16], ['node', ['OF'], 'City', 'countrycode', 'eg'], ['node', ['OF'], 'City', 'accent_name', 'Aswan'], ['node', ['OF'], 'City', 'location', [32.898888899999996, 24.0875]], ['node', ['OF'], 'City', 'name', 'aswan'], ['node', ['OF'], 'City', 'longitude', 32.898888899999996], ['node', ['OF'], 'City', 'latitude', 24.0875], ['node', ['OF'], 'City', 'country', 'Egypt'], ['node', ['OF'], 'City', 'population', 241282.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.139460234581], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.309102880979], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.2343926615229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.30992106867853], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.4738043471621], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.6815053682407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.1766494875258], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4168357886873], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.6859595933917], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.9113870097148], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.7263031184447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.3818728160173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738 E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 531.094660748449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.6726075866235], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 46.4], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.45], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 10.76], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.4], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 303175127598.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 9.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 238560.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'EGY'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 20], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 12.5], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 15.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 836000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 0.1], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 3.33], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 30.802498], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'EGP'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 42895824.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 288.57], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 100388073.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 18.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 1001450.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 106.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '818'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 26.38], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 37.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 35.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 26.820553], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Modern Standard Arabic'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 71.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 3.8], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 62.0]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: timezone:2,icao:HESN\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: tz_database_timezone,distCity,latitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,distCity,latitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('HESN', ['country', 'longitude', 'altitude'], ['iata', 'runwaysLength', 'runways'], {'difficultyProp': 'easyhard', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.611700058, 22.3759994507]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 616.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Abu Simbel'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ABS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HEBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Abu Simbel Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2457.370449965094], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.611700058], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.3759994507], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1127], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.40559959411621, 30.12190055847168]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12358.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HECA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16990.960849098446], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.40559959411621], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 30.12190055847168], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1128], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Riyadh'], ['node', ['ROUTE_TO'], 'Airport', 'location', [39.156502, 21.6796]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 48.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12904.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Saudi Arabia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jeddah'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JED'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OEJN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'King Abdulaziz International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19233.42631403247], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 39.156502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.6796], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2072], ['node', ['OF'], 'City', 'region', 16], ['node', ['OF'], 'City', 'countrycode', 'eg'], ['node', ['OF'], 'City', 'accent_name', 'Aswan'], ['node', ['OF'], 'City', 'location', [32.898888899999996, 24.0875]], ['node', ['OF'], 'City', 'name', 'aswan'], ['node', ['OF'], 'City', 'longitude', 32.898888899999996], ['node', ['OF'], 'City', 'latitude', 24.0875], ['node', ['OF'], 'City', 'country', 'Egypt'], ['node', ['OF'], 'City', 'population', 241282.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.139460234581], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.309102880979], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.2343926615229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.30992106867853], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.4738043471621], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.6815053682407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.1766494875258], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4168357886873], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.6859595933917], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.9113870097148], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.7263031184447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.3818728160173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738 E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 531.094660748449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.6726075866235], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 46.4], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.45], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 10.76], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.4], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 303175127598.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 9.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 238560.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'EGY'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 20], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 12.5], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 15.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 836000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 0.1], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 3.33], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 30.802498], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'EGP'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 42895824.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 288.57], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 100388073.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 18.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 1001450.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 106.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '818'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 26.38], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 37.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 35.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 26.820553], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Modern Standard Arabic'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 71.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 3.8], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 62.0]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:11161.0,iata:ASW,runways:1.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: country,longitude,altitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: country,longitude,altitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('HESN', ['runways', 'runwaysLength', 'iata'], ['altitude', 'tz_database_timezone', 'source'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.611700058, 22.3759994507]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 616.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Abu Simbel'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ABS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HEBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Abu Simbel Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2457.370449965094], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.611700058], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.3759994507], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1127], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.40559959411621, 30.12190055847168]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12358.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HECA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16990.960849098446], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.40559959411621], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 30.12190055847168], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1128], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Riyadh'], ['node', ['ROUTE_TO'], 'Airport', 'location', [39.156502, 21.6796]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 48.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12904.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Saudi Arabia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jeddah'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JED'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OEJN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'King Abdulaziz International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19233.42631403247], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 39.156502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.6796], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2072], ['node', ['OF'], 'City', 'region', 16], ['node', ['OF'], 'City', 'countrycode', 'eg'], ['node', ['OF'], 'City', 'accent_name', 'Aswan'], ['node', ['OF'], 'City', 'location', [32.898888899999996, 24.0875]], ['node', ['OF'], 'City', 'name', 'aswan'], ['node', ['OF'], 'City', 'longitude', 32.898888899999996], ['node', ['OF'], 'City', 'latitude', 24.0875], ['node', ['OF'], 'City', 'country', 'Egypt'], ['node', ['OF'], 'City', 'population', 241282.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.139460234581], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.309102880979], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.2343926615229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.30992106867853], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.4738043471621], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.6815053682407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.1766494875258], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4168357886873], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.6859595933917], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.9113870097148], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.7263031184447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.3818728160173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738 E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 531.094660748449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.6726075866235], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 46.4], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.45], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 10.76], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.4], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 303175127598.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 9.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 238560.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'EGY'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 20], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 12.5], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 15.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 836000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 0.1], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 3.33], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 30.802498], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'EGP'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 42895824.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 288.57], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 100388073.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 18.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 1001450.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 106.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '818'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 26.38], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 37.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 35.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 26.820553], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Modern Standard Arabic'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 71.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 3.8], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 62.0]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: tz_database_timezone:Africa/Cairo,altitude:662.0,source:OurAirports\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,runwaysLength,iata. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,runwaysLength,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('HESN', ['iata', 'runwaysLength', 'runways'], ['runwaysWidth', 'commercial_flights'], {'difficultyProp': 'hardhard', 'entityDiff:': 'medium', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.611700058, 22.3759994507]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 616.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 9843.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Abu Simbel'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'ABS'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HEBL'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Abu Simbel Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 2457.370449965094], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.611700058], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 22.3759994507], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1127], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Africa/Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'location', [31.40559959411621, 30.12190055847168]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 382.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12358.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Egypt'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Cairo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'CAI'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '2'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'HECA'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Cairo International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 16990.960849098446], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 31.40559959411621], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 30.12190055847168], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 1128], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Riyadh'], ['node', ['ROUTE_TO'], 'Airport', 'location', [39.156502, 21.6796]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 48.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12904.333333333334], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Saudi Arabia'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Jeddah'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'JED'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '3'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 3.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'OEJN'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'King Abdulaziz International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 19233.42631403247], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 39.156502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.6796], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 2072], ['node', ['OF'], 'City', 'region', 16], ['node', ['OF'], 'City', 'countrycode', 'eg'], ['node', ['OF'], 'City', 'accent_name', 'Aswan'], ['node', ['OF'], 'City', 'location', [32.898888899999996, 24.0875]], ['node', ['OF'], 'City', 'name', 'aswan'], ['node', ['OF'], 'City', 'longitude', 32.898888899999996], ['node', ['OF'], 'City', 'latitude', 24.0875], ['node', ['OF'], 'City', 'country', 'Egypt'], ['node', ['OF'], 'City', 'population', 241282.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 537.139460234581], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 521.309102880979], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70 738'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 613.2343926615229], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 424.30992106867853], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 634.4738043471621], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 543.6815053682407], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 653.1766494875258], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 215766.06290285522], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 541.4168357886873], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 611.6859595933917], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 470.9113870097148], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '15893'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'M4'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 652.7263031184447], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 699624.2109321393], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 547.3818728160173], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '2143'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '738 E70'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'MS'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 531.094660748449], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 698027.1802185046], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 502.6726075866235], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '3754'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', '320'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'XY'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 46.4], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 0.45], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 10.76], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 0.4], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 303175127598.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 9.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 238560.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Cairo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'EGY'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 44.4], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 20], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 12.5], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 15.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 836000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 0.1], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 3.33], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 30.802498], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'EGP'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 42895824.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 288.57], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 100388073.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 18.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 1001450.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'EG'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 106.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '818'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 26.38], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 37.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 35.2], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 26.820553], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Modern Standard Arabic'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Africa'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 71.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 3.8], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Egypt'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 62.0]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysWidth:148.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: iata,runwaysLength,runways. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runwaysLength,runways. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VVCA', ['timezone', 'latitude', 'source'], ['longitude', 'type', 'name'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.80699920654295, 21.221200942993164]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hanoi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVNB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Noi Bai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 21384.714222013317], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.80699920654295], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.221200942993164], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3199], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.652000427, 10.8187999725]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11237.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ho Chi Minh City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SGN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVTS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Tan Son Nhat International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7824.911335951555], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.652000427], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 10.8187999725], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3205], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.9540780239038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.13057105961815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 643.9146642327269], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.1083148915934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: type:airport,name:Chu Lai International Airport,longitude:108.706001282\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: timezone,latitude,source. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,latitude,source. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VVCA', ['tz_database_timezone', 'country', 'longitude'], ['runwaysLength', 'runwaysWidth', 'runways'], {'difficultyProp': 'easyhard', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.80699920654295, 21.221200942993164]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hanoi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVNB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Noi Bai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 21384.714222013317], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.80699920654295], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.221200942993164], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3199], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.652000427, 10.8187999725]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11237.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ho Chi Minh City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SGN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVTS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Tan Son Nhat International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7824.911335951555], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.652000427], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 10.8187999725], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3205], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.9540780239038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.13057105961815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 643.9146642327269], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.1083148915934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:10007.0,runways:1.0,runwaysWidth:148.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: tz_database_timezone,country,longitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: tz_database_timezone,country,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VVCA', ['runwaysLength', 'iata', 'runwaysWidth'], ['tz_database_timezone', 'timezone', 'location'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.80699920654295, 21.221200942993164]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hanoi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVNB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Noi Bai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 21384.714222013317], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.80699920654295], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.221200942993164], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3199], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.652000427, 10.8187999725]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11237.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ho Chi Minh City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SGN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVTS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Tan Son Nhat International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7824.911335951555], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.652000427], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 10.8187999725], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3205], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.9540780239038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.13057105961815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 643.9146642327269], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.1083148915934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: tz_database_timezone:Asia/Saigon,location:[108.706001282, 15.4033002853],timezone:7\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runwaysLength,iata,runwaysWidth. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VVCA', ['runways', 'iata', 'runwaysWidth'], ['runwaysLength', 'commercial_flights'], {'difficultyProp': 'hardhard', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [105.80699920654295, 21.221200942993164]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 39.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11482.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Hanoi'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'HAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVNB'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Noi Bai International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 21384.714222013317], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 105.80699920654295], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 21.221200942993164], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3199], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Asia/Saigon'], ['node', ['ROUTE_TO'], 'Airport', 'location', [106.652000427, 10.8187999725]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 33.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11237.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Vietnam'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Ho Chi Minh City'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SGN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '7'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VVTS'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Tan Son Nhat International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 7824.911335951555], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 106.652000427], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 10.8187999725], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3205], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 576.9540780239038], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 479.13057105961815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 643.9146642327269], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 560.1083148915934], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 716380.118295195], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 556786.5798945981], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5309'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'AT7'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'VN']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:10007.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,iata,runwaysWidth. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VRMT', ['icao', 'timezone', 'city'], ['dst', 'type'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.52909851074219, 4.191830158233644]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11155.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Male'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MLE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Malé International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4275.925444375213], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.52909851074219], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 4.191830158233644], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3156], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.155602, -0.693342]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Gan Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Gan International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.155602], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -0.693342], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 165.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6183], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5251548537562], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.81569168212377], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3 DH2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH2 DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: dst:U,type:airport\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: icao,timezone,city. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: icao,timezone,city. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VRMT', ['source', 'altitude', 'longitude'], ['runwaysLength', 'iata', 'runways'], {'difficultyProp': 'easyhard', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.52909851074219, 4.191830158233644]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11155.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Male'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MLE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Malé International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4275.925444375213], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.52909851074219], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 4.191830158233644], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3156], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.155602, -0.693342]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Gan Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Gan International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.155602], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -0.693342], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 165.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6183], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5251548537562], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.81569168212377], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3 DH2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH2 DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:4003.0,iata:KDM,runways:1.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: source,altitude,longitude. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: source,altitude,longitude. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VRMT', ['runways', 'iata', 'commercial_flights'], ['city', 'tz_database_timezone', 'altitude'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.52909851074219, 4.191830158233644]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11155.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Male'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MLE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Malé International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4275.925444375213], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.52909851074219], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 4.191830158233644], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3156], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.155602, -0.693342]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Gan Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Gan International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.155602], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -0.693342], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 165.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6183], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5251548537562], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.81569168212377], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3 DH2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH2 DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: tz_database_timezone:Indian/Maldives,altitude:2.0,city:Kaadedhdhoo\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,iata,commercial_flights. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,iata,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('VRMT', ['runwaysLength', 'commercial_flights', 'runwaysWidth'], ['runways', 'iata'], {'difficultyProp': 'hardhard', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.52909851074219, 4.191830158233644]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 11155.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Male'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'MLE'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Malé International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 4275.925444375213], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.52909851074219], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 4.191830158233644], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 197.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 3156], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Indian/Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'location', [73.155602, -0.693342]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 6.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'U'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 12000.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Maldives'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Gan Island'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'GAN'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '5'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'VRMG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Gan International Airport'], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 73.155602], ['node', ['ROUTE_TO'], 'Airport', 'latitude', -0.693342], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 165.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 6183], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 572.5251548537562], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 469.81569168212377], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3 DH2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 416520.32084736926], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH2 DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 132702.18670993272], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '13108'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH3'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'Q2']], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: iata:KDM,runways:1.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runwaysLength,commercial_flights,runwaysWidth. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runwaysLength,commercial_flights,runwaysWidth. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ENSD', ['city', 'country', 'name'], ['latitude', 'distCity'], {'difficultyProp': 'easyeasy', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 212806], ['node', ['ROUTE_TO'], 'Airport', 'location', [11.0502, 60.121]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 681.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10744.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OSL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENGM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Oslo Lufthavn'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28218.144603852397], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 11.0502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 60.121], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 644], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 5174], ['node', ['ROUTE_TO'], 'Airport', 'location', [7.13778, 61.156101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1633.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3150.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sogndal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SOG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENSG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sogndal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 317716.33085863624], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 7.13778], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 61.156101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 5586], ['node', ['OF'], 'City', 'region', 15], ['node', ['OF'], 'City', 'countrycode', 'no'], ['node', ['OF'], 'City', 'accent_name', 'Sandane'], ['node', ['OF'], 'City', 'location', [6.216667, 61.766667000000005]], ['node', ['OF'], 'City', 'name', 'sandane'], ['node', ['OF'], 'City', 'longitude', 6.216667], ['node', ['OF'], 'City', 'latitude', 61.766667000000005], ['node', ['OF'], 'City', 'country', 'Norway'], ['node', ['OF'], 'City', 'population', 2137.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.657265072687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.587144131525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.5232117486168], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.0063084188673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.7076397444575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 447.3675880480815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 63.8], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.92], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 3.35], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 1.78], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 403336363636.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 41023.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'NOR'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 47], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.9], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 154.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 23000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.2], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.56], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 8.468946], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'NOK'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 4418218.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 120.27], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 5347896.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 2.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 323802.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '578'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.4], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 150.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 82.0], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 60.472024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Norwegian'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 2.7], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 14.3]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: distCity:9149.113244809485,latitude:61.830001831055\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: city,country,name. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: city,country,name. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ENSD', ['timezone', 'icao', 'country'], ['runwaysWidth', 'iata', 'runwaysLength'], {'difficultyProp': 'easyhard', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 212806], ['node', ['ROUTE_TO'], 'Airport', 'location', [11.0502, 60.121]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 681.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10744.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OSL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENGM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Oslo Lufthavn'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28218.144603852397], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 11.0502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 60.121], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 644], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 5174], ['node', ['ROUTE_TO'], 'Airport', 'location', [7.13778, 61.156101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1633.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3150.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sogndal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SOG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENSG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sogndal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 317716.33085863624], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 7.13778], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 61.156101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 5586], ['node', ['OF'], 'City', 'region', 15], ['node', ['OF'], 'City', 'countrycode', 'no'], ['node', ['OF'], 'City', 'accent_name', 'Sandane'], ['node', ['OF'], 'City', 'location', [6.216667, 61.766667000000005]], ['node', ['OF'], 'City', 'name', 'sandane'], ['node', ['OF'], 'City', 'longitude', 6.216667], ['node', ['OF'], 'City', 'latitude', 61.766667000000005], ['node', ['OF'], 'City', 'country', 'Norway'], ['node', ['OF'], 'City', 'population', 2137.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.657265072687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.587144131525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.5232117486168], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.0063084188673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.7076397444575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 447.3675880480815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 63.8], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.92], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 3.35], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 1.78], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 403336363636.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 41023.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'NOR'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 47], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.9], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 154.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 23000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.2], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.56], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 8.468946], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'NOK'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 4418218.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 120.27], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 5347896.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 2.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 323802.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '578'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.4], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 150.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 82.0], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 60.472024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Norwegian'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 2.7], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 14.3]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:3182.0,iata:SDN,runwaysWidth:98.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: timezone,icao,country. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: timezone,icao,country. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ENSD', ['iata', 'runways', 'commercial_flights'], ['latitude', 'dst', 'icao'], {'difficultyProp': 'hardeasy', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 212806], ['node', ['ROUTE_TO'], 'Airport', 'location', [11.0502, 60.121]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 681.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10744.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OSL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENGM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Oslo Lufthavn'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28218.144603852397], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 11.0502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 60.121], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 644], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 5174], ['node', ['ROUTE_TO'], 'Airport', 'location', [7.13778, 61.156101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1633.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3150.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sogndal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SOG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENSG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sogndal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 317716.33085863624], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 7.13778], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 61.156101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 5586], ['node', ['OF'], 'City', 'region', 15], ['node', ['OF'], 'City', 'countrycode', 'no'], ['node', ['OF'], 'City', 'accent_name', 'Sandane'], ['node', ['OF'], 'City', 'location', [6.216667, 61.766667000000005]], ['node', ['OF'], 'City', 'name', 'sandane'], ['node', ['OF'], 'City', 'longitude', 6.216667], ['node', ['OF'], 'City', 'latitude', 61.766667000000005], ['node', ['OF'], 'City', 'country', 'Norway'], ['node', ['OF'], 'City', 'population', 2137.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.657265072687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.587144131525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.5232117486168], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.0063084188673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.7076397444575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 447.3675880480815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 63.8], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.92], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 3.35], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 1.78], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 403336363636.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 41023.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'NOR'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 47], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.9], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 154.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 23000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.2], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.56], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 8.468946], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'NOK'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 4418218.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 120.27], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 5347896.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 2.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 323802.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '578'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.4], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 150.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 82.0], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 60.472024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Norwegian'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 2.7], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 14.3]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: dst:E,icao:ENSD,latitude:61.830001831055\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: iata,runways,commercial_flights. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: iata,runways,commercial_flights. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']}), ('ENSD', ['runways', 'commercial_flights', 'iata'], ['runwaysLength', 'runwaysWidth'], {'difficultyProp': 'hardhard', 'entityDiff:': 'hard', 'context': [['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 212806], ['node', ['ROUTE_TO'], 'Airport', 'location', [11.0502, 60.121]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 681.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 10744.5], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'OSL'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 2.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENGM'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Oslo Lufthavn'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 28218.144603852397], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 11.0502], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 60.121], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 148.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 644], ['node', ['ROUTE_TO'], 'Airport', 'tz_database_timezone', 'Europe/Oslo'], ['node', ['ROUTE_TO'], 'Airport', 'commercial_flights', 5174], ['node', ['ROUTE_TO'], 'Airport', 'location', [7.13778, 61.156101]], ['node', ['ROUTE_TO'], 'Airport', 'altitude', 1633.0], ['node', ['ROUTE_TO'], 'Airport', 'dst', 'E'], ['node', ['ROUTE_TO'], 'Airport', 'type', 'airport'], ['node', ['ROUTE_TO'], 'Airport', 'runwaysLength', 3150.0], ['node', ['ROUTE_TO'], 'Airport', 'country', 'Norway'], ['node', ['ROUTE_TO'], 'Airport', 'city', 'Sogndal'], ['node', ['ROUTE_TO'], 'Airport', 'iata', 'SOG'], ['node', ['ROUTE_TO'], 'Airport', 'timezone', '1'], ['node', ['ROUTE_TO'], 'Airport', 'source', 'OurAirports'], ['node', ['ROUTE_TO'], 'Airport', 'runways', 1.0], ['node', ['ROUTE_TO'], 'Airport', 'icao', 'ENSG'], ['node', ['ROUTE_TO'], 'Airport', 'name', 'Sogndal Airport'], ['node', ['ROUTE_TO'], 'Airport', 'distCity', 317716.33085863624], ['node', ['ROUTE_TO'], 'Airport', 'longitude', 7.13778], ['node', ['ROUTE_TO'], 'Airport', 'latitude', 61.156101], ['node', ['ROUTE_TO'], 'Airport', 'runwaysWidth', 98.0], ['node', ['ROUTE_TO'], 'Airport', 'airport_id', 5586], ['node', ['OF'], 'City', 'region', 15], ['node', ['OF'], 'City', 'countrycode', 'no'], ['node', ['OF'], 'City', 'accent_name', 'Sandane'], ['node', ['OF'], 'City', 'location', [6.216667, 61.766667000000005]], ['node', ['OF'], 'City', 'name', 'sandane'], ['node', ['OF'], 'City', 'longitude', 6.216667], ['node', ['OF'], 'City', 'latitude', 61.766667000000005], ['node', ['OF'], 'City', 'country', 'Norway'], ['node', ['OF'], 'City', 'population', 2137.0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 499.657265072687], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 512.587144131525], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 505.5232117486168], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 327745.30580121226], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 575.0063084188673], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'stops', 0], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'duration', 645.7076397444575], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'distance', 92915.21204473457], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'price', 447.3675880480815], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline_id', '5439'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'equipment', 'DH1'], ['edge', ['ROUTE_TO'], 'ROUTE_TO', 'airline', 'WF'], ['node', ['OF', 'City', 'IN'], 'Country', 'region', 'Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'labor_participation', 63.8], ['node', ['OF', 'City', 'IN'], 'Country', 'physicians', 2.92], ['node', ['OF', 'City', 'IN'], 'Country', 'unemployment', 3.35], ['node', ['OF', 'City', 'IN'], 'Country', 'gasoline_price', 1.78], ['node', ['OF', 'City', 'IN'], 'Country', 'gdp', 403336363636.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi_change', 2.2], ['node', ['OF', 'City', 'IN'], 'Country', 'capital', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'co2_emissions', 41023.0], ['node', ['OF', 'City', 'IN'], 'Country', 'iso_3166_2', 'ISO 3166-2:NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'largest_city', 'Oslo'], ['node', ['OF', 'City', 'IN'], 'Country', 'alpha3', 'NOR'], ['node', ['OF', 'City', 'IN'], 'Country', 'total_tax_rate', 36.2], ['node', ['OF', 'City', 'IN'], 'Country', 'calling_code', 47], ['node', ['OF', 'City', 'IN'], 'Country', 'tax_revenue', 23.9], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region_code', 154.0], ['node', ['OF', 'City', 'IN'], 'Country', 'armed_forces', 23000.0], ['node', ['OF', 'City', 'IN'], 'Country', 'name', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'forested_area', 33.2], ['node', ['OF', 'City', 'IN'], 'Country', 'fertility_rate', 1.56], ['node', ['OF', 'City', 'IN'], 'Country', 'longitude', 8.468946], ['node', ['OF', 'City', 'IN'], 'Country', 'currency_code', 'NOK'], ['node', ['OF', 'City', 'IN'], 'Country', 'urban_population', 4418218.0], ['node', ['OF', 'City', 'IN'], 'Country', 'cpi', 120.27], ['node', ['OF', 'City', 'IN'], 'Country', 'population', 5347896.0], ['node', ['OF', 'City', 'IN'], 'Country', 'infant_mortality', 2.1], ['node', ['OF', 'City', 'IN'], 'Country', 'land_area', 323802.0], ['node', ['OF', 'City', 'IN'], 'Country', 'code', 'NO'], ['node', ['OF', 'City', 'IN'], 'Country', 'education_primary', 100.3], ['node', ['OF', 'City', 'IN'], 'Country', 'country_code', '578'], ['node', ['OF', 'City', 'IN'], 'Country', 'birth_rate', 10.4], ['node', ['OF', 'City', 'IN'], 'Country', 'maternal_mortality', 2.0], ['node', ['OF', 'City', 'IN'], 'Country', 'region_code', 150.0], ['node', ['OF', 'City', 'IN'], 'Country', 'education_tertiary', 82.0], ['node', ['OF', 'City', 'IN'], 'Country', 'intermediate_region', ''], ['node', ['OF', 'City', 'IN'], 'Country', 'latitude', 60.472024], ['node', ['OF', 'City', 'IN'], 'Country', 'language', 'Norwegian'], ['node', ['OF', 'City', 'IN'], 'Country', 'sub_region', 'Northern Europe'], ['node', ['OF', 'City', 'IN'], 'Country', 'life_expectancy', 82.8], ['node', ['OF', 'City', 'IN'], 'Country', 'agricultural_land', 2.7], ['node', ['OF', 'City', 'IN'], 'Country', 'name2', 'Norway'], ['node', ['OF', 'City', 'IN'], 'Country', 'health_expenditure', 14.3]], 'input': '\n    Here is the entity extracted from the database that you should seek information about:\n\n        input: runwaysLength:3182.0,runwaysWidth:98.0\n\n        context: [NO_CONTEXT_ADDED].\n    ', 'prompts': ['You enrich database entities by identifying their real-world counterpart and adding the following missing properties: runways,commercial_flights,iata. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio\n\n\n## Input\niata: "REC"\nstate: Pernambuco\n\n## Output\n\nproperty,value\nname, Aeroporto Internacional do Recife/Guararapes–Gilberto Freyre\ncity, Recife\n\n## Input\niata: "EZE"\ncountry: "Argentina""\ntype: "Passengers\n\n## Output\n\nproperty,value\nname, Ministro Pistarini International Airport\ncity, Buenos Aires\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text or tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding verified values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly\n\n## Input\n\niata: "CDG"\n\n## Output\n\nproperty,value\nname, Charles de Gaulle Airport\ncity, Paris\n\n\n## Input\niata: "MCZ"\nrunways: 1\n\n## Output\n\nproperty,value\nname, Zumbi dos Palmares Airport\ncity, Maceio', '# Identity\n\nYou are an assistant that enrichs entities from a database by identifying the corresponding real-world entity and adding values for specific properties. These attributes will be transformed into property:value pairs in a property-graph database.\n\n# Instructions\n1. From the input node passed by the user (property:value pairs), infer the most likely real-world entity and its entity class (Person, Organization, Location, Product, Country). The entity can be either nodes or relationship between two real-world entities.\n2. Find trustworthy and recent values for the following properties of this entity: runways,commercial_flights,iata. If you are not confident about the values, do not include them.\n3. Do not include any other property and do not include information that already exists in the input.\n\n# Output format\n\nOut only CSV with header `property,value`. Do not output any other text besides the properties and values or csv tags\nThese will be related only to properties of nodes.\nIf no facts can be verified, output only the header row.\n\n\n# Final instruction\n\nYou need to think step by step in your system. First, identify entity class. Then, search each asked property using web search or your own knowlege base, adding values to CSV. Stop when no more high-confidence facts available.\n\n# Examples\n\n## Input\n\niata: "ORY"\ncountry: "France"\n\n## Output\n\nproperty,value\nname, Paris-Orly Airport\ncity, Orly']})]

In [ ]:
# Sending to batch api
# You need to change the model name to the one you want to use, and also the file name and config as needed.


from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)


uploaded_file = client.files.upload(
    file="gemini_batch_file.jsonl",
    config=types.UploadFileConfig(display_name='my-batch-requests', mime_type='jsonl')
)

inline_requests = inline
# Rodar os modelos diferentes depois..
batch_job = client.batches.create(
    model="gemini-2.5-pro",
    src=uploaded_file.name, 
    config={
        "display_name": "entity-enrichment-batch-contextpro"
    }
)

print(f"Created batch job: {batch_job.name}")

Created batch job: batches/lmzk04hs2ackyimssznw073ls52lmljmzi54


In [ ]:
import os
import json
import csv
from google import genai
# This code is responsible for checking the status of the batch job and downloading the results once it's completed.
# You can run this code in a separate script or after some time to check the results.

# --------------------------------------------------
# Config
OUTPUT_DIR = "context_gemini_pro_batch_results3_csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_CSV = os.path.join(OUTPUT_DIR, "usage_summary.csv")

client = genai.Client(api_key=API_KEY)

job_name = "batches/nridlzv6lukk945jthxz7gjdp5jtqgta5blb"
batch_job = client.batches.get(name=job_name)

# --------------------------------------------------
# Check job status
if batch_job.state.name != "JOB_STATE_SUCCEEDED":
    raise RuntimeError(f"Batch job failed: {batch_job.state.name}")

print("Batch job succeeded.")

# --------------------------------------------------
# Download batch result file
content = client.files.download(file=batch_job.dest.file_name)

# --------------------------------------------------
# Helper: flatten usageMetadata
def flatten_usage(usage: dict) -> dict:
    flat = {}
    for k, v in usage.items():
        if isinstance(v, dict):
            for sub_k, sub_v in v.items():
                flat[f"{k}_{sub_k}"] = sub_v
        else:
            flat[k] = v
    return flat

rows = []
all_columns = {"key"}

# --------------------------------------------------
# Parse JSONL
for line in content.decode("utf-8").splitlines():
    record = json.loads(line)

    key = record.get("key")
    response = record.get("response")
    error = record.get("error")

    if error or not response:
        continue

    usage = response.get("usageMetadata")
    if not usage:
        continue

    flat_usage = flatten_usage(usage)

    row = {"key": key, **flat_usage}
    rows.append(row)
    all_columns.update(row.keys())

# --------------------------------------------------
# Write CSV
columns = sorted(all_columns)

with open("output_tokens_gemini_pro.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=columns)
    writer.writeheader()
    writer.writerows(rows)

print(f"Saved Gemini usage CSV: {OUTPUT_CSV}")


Batch job succeeded.
Saved Gemini usage CSV: context_gemini_pro_batch_results3_csv/usage_summary.csv


GPT

In [ ]:
import json

def build_gpt_batch_requests(data, runs=5):

    requests = []
    request_id = 0

    for (
        entity_id,
        hidden_props,
        given_props,
        meta
    ) in data:
        entity_input = meta.get("input", "").strip()
        prompts = meta.get("prompts", [])

        for prompt_idx, prompt_text in enumerate(prompts):
            for run in range(runs):
                request = {
                    "custom_id": f"context_gpt_mini_entity_{entity_id}_p{prompt_idx}_run{run+1}_{meta['difficultyProp']}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": 'gpt-5',
                        "messages": [
                            {
                                "role": "system",
                                "content": prompt_text.strip()
                            },
                            {
                                "role": "user",
                                "content": entity_input
                            }
                        ],
                    }
                }
                
                requests.append(request)
                request_id += 1

    return requests


# Build requests
gpt_requests = build_gpt_batch_requests(results)

# Save JSONL
with open("gpt_batch_requests.jsonl", "w", encoding="utf-8") as f:
    for req in gpt_requests:
        f.write(json.dumps(req) + "\n")

print(f"Saved {len(gpt_requests)} GPT batch requests.")


Saved 1620 GPT batch requests.


In [250]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

batch_input_file = client.files.create(
    file=open("gpt_batch_requests.jsonl", "rb"),
    purpose="batch"
)

print("Uploaded file ID:", batch_input_file.id)

batch_job = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h"  # or "12h"
)

print("Batch job ID:", batch_job.id)
print("Initial status:", batch_job.status)


Uploaded file ID: file-2dMVM2LjtgeNHJ2PoQZsGQ
Batch job ID: batch_6944a28837bc8190b2d190452b212143
Initial status: validating


In [ ]:
import os
import json
import csv
from openai import OpenAI

# --------------------------------------------------
# Config
OUTPUT_DIR = "context_gpt_batch_results_csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_CSV = os.path.join(OUTPUT_DIR, "usage_summary.csv")

client = OpenAI(api_key=OPENAI_API_KEY)

batch_id = "batch_69436ae94e0881909a767c9ce4c906e0"

# --------------------------------------------------
# Retrieve batch job
batch_job = client.batches.retrieve(batch_id)

if batch_job.status != "completed":
    raise RuntimeError(f"Batch job failed: {batch_job.status}")

# --------------------------------------------------
# Download output file
output_file_id = batch_job.output_file_id
content = client.files.content(output_file_id).read()


def flatten_usage(usage: dict) -> dict:
    flat = {}
    for k, v in usage.items():
        if isinstance(v, dict):
            for sub_k, sub_v in v.items():
                flat[f"{k}_{sub_k}"] = sub_v
        else:
            flat[k] = v
    return flat

rows = []
all_columns = set(["key"])

# --------------------------------------------------
# Parse JSONL
for line in content.decode("utf-8").splitlines():
    record = json.loads(line)

    key = record.get("custom_id")
    response = record.get("response")
    error = record.get("error")

    if error or not response:
        continue

    usage = response["body"].get("usage")
    if not usage:
        continue

    flat_usage = flatten_usage(usage)

    row = {"key": key, **flat_usage}
    rows.append(row)
    all_columns.update(row.keys())

# --------------------------------------------------
# Write CSV
columns = sorted(all_columns)

with open("output_tokens_gpt.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=columns)
    writer.writeheader()
    writer.writerows(rows)

print(f"Saved usage CSV: {OUTPUT_CSV}")


Saved usage CSV: context_gpt_batch_results_csv/usage_summary.csv


Anthropic Claude

In [ ]:
def build_claude_inline_requests(
    data,
    n_runs=5,
    model="claude-sonnet-4-5",
    max_tokens=1024
):
    requests_by_entity = {}
    reqs = []

    for (
        entity_id,
        hidden_props,
        given_props,
        meta
    ) in data:

        entity_input = meta.get("input", "").strip()
        prompts = meta.get("prompts", [])

        entity_requests = []

        for prompt_idx, prompt_text in enumerate(prompts):
            for run in range(n_runs):

                request_key = f"context_sonnet_entity_{entity_id}_p{prompt_idx}_run{run+1}_{meta['difficultyProp']}"

                # Anthropic best practice: single user message
                user_text = (
                    prompt_text.strip().replace("\n", " ")
                    + "\n\n"
                    + entity_input
                )

                params = {
                    "model": model,
                    "max_tokens": max_tokens,
                    "messages": [
                        {
                            "role": "user",
                            "content": [
                                {
                                    "type": "text",
                                    "text": user_text
                                }
                            ]
                        }
                    ]
                }

                entity_requests.append({
                    "custom_id": request_key, 
                    "params": params
                })

        requests_by_entity[entity_id] = entity_requests
        reqs.append(entity_requests)

    return reqs

claude_req = build_claude_inline_requests(results)
print(claude_req[:10])

[[{'custom_id': 'context_sonnet_entity_PAKK_p0_run1_easyeasy', 'params': {'model': 'claude-sonnet-4-5', 'max_tokens': 1024, 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'You enrich database entities by identifying their real-world counterpart and adding the following missing properties: location,source,dst. Using the input node (property:value pairs), infer the most likely real-world entity and its class (Person, Organization, Location, Product, Country). Retrieve trustworthy, recent values for these properties. Only include values you are confident about and that are not already in the input. Return CSV only with header property,value.\n\nHere is the entity extracted from the database that you should seek information about:\n\n        input: altitude:154.0,city:Koyuk\n\n        context: [NO_CONTEXT_ADDED].'}]}]}}, {'custom_id': 'context_sonnet_entity_PAKK_p0_run2_easyeasy', 'params': {'model': 'claude-sonnet-4-5', 'max_tokens': 1024, 'messages': [{'role': 'user',

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)

inline_requests = []
for entity_requests in claude_req:
    inline_requests.extend(entity_requests)

batch = client.messages.batches.create(
    requests=inline_requests
)

print("Created Claude batch:", batch.id)


Created Claude batch: msgbatch_01BXXF2PXcE75jYZvNNxgWQD


In [ ]:
import anthropic
OUTPUT_DIR = "context_claude_sonnet_batch_results_csv"
client = anthropic.Anthropic()

for resultClaude in client.messages.batches.results(
    "msgbatch_01BXXF2PXcE75jYZvNNxgWQD",
):
    custom_id = resultClaude.custom_id

    try:
        # Claude messages always return a list of content blocks
        text = resultClaude.result.message.content[0].text
    except Exception:
        text = str(resultClaude.result.message.content[0].text)

    filename = f"{custom_id}.csv"
    output_path = os.path.join(OUTPUT_DIR, filename)

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"Saved: {output_path}")  

Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p0_run1_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p0_run2_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p0_run3_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p0_run4_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p0_run5_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p1_run1_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p1_run2_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p1_run3_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p1_run4_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/context_sonnet_entity_PAKK_p1_run5_easyeasy.csv
Saved: context_claude_sonnet_batch_results_csv/con

In [ ]:
import os
import csv
import anthropic


OUTPUT_DIR = "context_claude_sonnet_batch_results_csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_CSV = os.path.join(OUTPUT_DIR, "usage_summary.csv")

client = anthropic.Anthropic()

batch_id = "msgbatch_016V8MDdriiUxEHXFrTuYzZw"


def flatten_usage(usage: dict) -> dict:
    flat = {}
    for k, v in usage.items():
        if isinstance(v, dict):
            for sub_k, sub_v in v.items():
                flat[f"{k}_{sub_k}"] = sub_v
        else:
            flat[k] = v
    return flat

rows = []
all_columns = {"key"}

for result in client.messages.batches.results(batch_id):
    custom_id = result.custom_id

    if result.result.type != "succeeded":
        continue

    message = result.result.message


    usage = getattr(message, "usage", None)
    if not usage:
        continue

    # Convert usage object → dict
    usage_dict = usage.model_dump()

    flat_usage = flatten_usage(usage_dict)

    row = {"key": custom_id, **flat_usage}
    rows.append(row)
    all_columns.update(row.keys())

columns = sorted(all_columns)

with open("output_tokens_claude_sonnet.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=columns)
    writer.writeheader()
    writer.writerows(rows)

print(f"Saved Claude usage CSV: {OUTPUT_CSV}")


Saved Claude usage CSV: context_claude_sonnet_batch_results_csv/usage_summary.csv


## Data Integration

Now, with the results from each provider, the following code will be responsible for the voting function.

In [ ]:
import os
import re
import csv
from io import StringIO
from collections import defaultdict, Counter
import pandas as pd

# --------------------------------------------------
# Data integration logic
# --------------------------------------------------
def data_integration(values: list):
    if not values:
        return None

    # Try numeric integration
    try:
        numeric_vals = [float(v) for v in values]
        return sum(numeric_vals) / len(numeric_vals)
    except ValueError:
        # Textual → majority vote
        most_common, _ = Counter(values).most_common(1)[0]
        return most_common


# --------------------------------------------------
# Majority vote per test
# --------------------------------------------------
def calculate_majority_vote(grouped_votes):
    final_results = {}

    for test_name, prop_map in grouped_votes.items():
        result = {}
        for prop, values in prop_map.items():
            integrated_value = data_integration(values)
            if integrated_value is not None:
                result[prop] = integrated_value

        final_results[test_name] = result

    return final_results


# --------------------------------------------------
# Parse filename
# gemini25pro_entity_ENSD_p8_run2_hardhard.csv
# → ENSD_p8_hardhard
# --------------------------------------------------
FILENAME_PATTERN = re.compile(
    r"""
    ^\['
    .*?_entity_
    (?P<entity>[A-Z0-9]+)_
    (?P<prompt>p\d+)_run\d+_
    (?P<difficulty>[a-zA-Z]+)
    '\]\.csv$
    """,
    re.VERBOSE
)

def extract_test_name(filename: str):
    match = FILENAME_PATTERN.match(filename)
    if not match:
        return None

    entity = match.group("entity")
    prompt = match.group("prompt")
    difficulty = match.group("difficulty")

    return f"{entity}_{prompt}_{difficulty}"


# --------------------------------------------------
# Read CSVs and group runs
# --------------------------------------------------
def organize_votes(directory: str):
    grouped_votes = defaultdict(lambda: defaultdict(list))

    for filename in os.listdir(directory):
        if not filename.endswith(".csv"):
            continue

        test_name = extract_test_name(filename)
        if test_name is None:
            continue

        file_path = os.path.join(directory, filename)

        with open(file_path, encoding="utf-8") as f:
            lines = f.read().splitlines()

            # Remove markdown fences if present
            if lines and lines[0].strip().lower().startswith("```csv") or lines[0].strip().lower().startswith("```"):
                lines = lines[1:]
            if lines and lines[-1].strip() == "```":
                lines = lines[:-1]

            csv_content = "\n".join(lines)
            reader = csv.DictReader(StringIO(csv_content))

            for row in reader:
                prop = row.get("property")
                val = row.get("value")

                if not prop or not val:
                    continue

                grouped_votes[test_name][prop.strip()].append(val.strip())

    return calculate_majority_vote(grouped_votes)


# --------------------------------------------------
# Convert results to DataFrame
# --------------------------------------------------
def build_dataframe(results, output_csv="contextgptbig_airports_majority_vote.csv"):
    all_properties = set()
    for info in results.values():
        all_properties.update(info.keys())

    columns = ["test"] + sorted(all_properties)
    rows = []

    for test, info in results.items():
        row = {"test": test}
        row.update(info)
        rows.append(row)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_csv, index=False, encoding="utf-8")

    return df


# --------------------------------------------------
# Run
# --------------------------------------------------

directory = "context_gpt_batch_results_csv"

results = organize_votes(directory)
df = build_dataframe(results)

print(df)


                 test altitude city class commercial_flights         country  \
0    ENSD_p5_hardeasy      NaN  NaN   NaN               true             NaN   
1    KEUG_p0_hardhard      NaN  NaN   NaN               true             NaN   
2    ZUBJ_p5_hardeasy      NaN  NaN   NaN                NaN             NaN   
3    VEGT_p1_hardeasy      NaN  NaN   NaN               true             NaN   
4    YPPH_p1_hardeasy      NaN  NaN   NaN                NaN             NaN   
..                ...      ...  ...   ...                ...             ...   
215  VVCA_p0_easyhard      NaN  NaN   NaN                NaN           Nepal   
216  VRMT_p2_hardeasy      NaN  NaN   NaN               true             NaN   
217  VVCA_p5_easyhard      NaN  NaN   NaN                NaN  United Kingdom   
218  PAKK_p1_easyeasy      NaN  NaN   NaN                NaN             NaN   
219  VVCA_p8_easyhard      NaN  NaN   NaN                NaN           Nepal   

    distCity  dst iata icao  latitude  